## Georgia 2022 Primary and Primary Runoff Election Returns and Boundaries

### Sections
- <a href="#join">Read in Input Files and Join</a><br>
- <a href="#shp">Modify Precinct Boundaries and Names</a><br>
- <a href="#maup">Join with Election Returns</a><br>
- <a href="#splits">Split Precincts</a><br>
- <a href="#check">Vote Total Checks</a><br>
- <a href="#exp">Export Cleaned Precinct-Level Datasets</a><br>

#### Sources
Note: This file was created after the 2022 Georgia primary and primary runoff precinct-level election results files, those files are used as an input to this notebook and the processing code can be found on the [RDH github.](https://github.com/nonpartisan-redistricting-datahub/pber_collection/tree/main/GA/2022)

- [RDH Georgia 2022 Primary Election Results, Precinct Level](https://redistrictingdatahub.org/dataset/georgia-2022-primary-election-precinct-level-results/) 
- [RDH Georgia 2022 Primary Runoff Election Results, Precinct Level](https://redistrictingdatahub.org/dataset/georgia-2022-primary-election-runoff-precinct-level-results/)
- [RDH Georgia 2022 General Election Results and Boundaries, Precinct Level](https://redistrictingdatahub.org/dataset/georgia-2022-general-election-precinct-level-results-and-boundaries/)
- [RDH Georgia 2020 General/Special/Runoff Election Results and Boundaries, Precinct Level](https://redistrictingdatahub.org/dataset/georgia-2020-general-election-precinct-and-election-results-extended/)
-[2021 Georgia Congressional Districts](https://redistrictingdatahub.org/dataset/2021-georgia-congressional-districts-approved-plan/)
-[2021 Adopted State Senate Plan](https://redistrictingdatahub.org/dataset/2021-georgia-state-senate-districts-approved-plan/)
-[2021 Adopted State House Plan](https://redistrictingdatahub.org/dataset/2021-georgia-state-house-of-representatives-districts-approved-plan/)


In [1]:
import pandas as pd
import geopandas as gp
import os
import numpy as np
import re
from collections import Counter
from pber_helper_GA import *
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [2]:
# Temporarily ignore warning messages for splits function
import warnings

warnings.filterwarnings("ignore")

<p><a name="join"></a></p>

### Read in input files

In [3]:
# District Shapefiles
#2022 GA Congressional plan
ga_cong = gp.read_file("./raw-from-source/ga_cong_adopted_2021/CONGRESS-PROP1-2021-shape.shp")
#2022 GA State Senate Plan
ga_sldu = gp.read_file("./raw-from-source/ga_sldu_adopted_2021/SENATE-PROP1-2021-Shape.shp")
#2022 GA State House Plan
ga_sldl = gp.read_file("./raw-from-source/ga_sldl_adopted_2021/HOUSE-PROP1-2021-shape.shp")

In [4]:
# Read in Primary Election Returns
ga_prim_er = pd.read_csv('./raw-from-source/ga_22_primary_prec/ga_22_primary_prec.csv')
# Read in Primary Runoff Election Returns
ga_prim_runoff_er = pd.read_csv('./raw-from-source/ga_22_primary_runoff_prec/ga_22_primary_runoff_prec.csv')
# Read in GA 22 General Shapefile
ga_gen_shp = gp.read_file('./raw-from-source/ga_2022_gen_prec/ga_2022_gen_prec_no_splits/ga_2022_gen_prec_no_splits.shp')
# # subset to just columns of interest
# ga_gen_shp = ga_gen_shp[['UNIQUE_ID', 'county', 'precinct', 'geometry']]
# Read in GA 20 General Shapefile
ga_gen_20_shp = gp.read_file('./raw-from-source/ga_gen_20_prec/ga_gen_20_st_prec.shp')

In [5]:
#Set unique ID to include County name
ga_prim_er['UNIQUE_ID'] = ga_prim_er['county'] +'-:-'+ ga_prim_er['precinct']
ga_prim_runoff_er['UNIQUE_ID'] = ga_prim_runoff_er['county'] +'-:-'+ ga_prim_runoff_er['precinct']
ga_gen_shp['UNIQUE_ID'] = ga_gen_shp['county'] +'-:-'+ ga_gen_shp['precinct']

#### Join Primary and Primary Runoff Election Returns

In [6]:
# check if all unique_ids match up
set(ga_prim_er['UNIQUE_ID']) == set(ga_prim_runoff_er['UNIQUE_ID'])

False

In [7]:
# Look at further name anomalies
runoff_names = sorted(list(set(ga_prim_runoff_er['UNIQUE_ID']) - set(ga_prim_er['UNIQUE_ID'])))
prim_names = sorted(list(set(ga_prim_er['UNIQUE_ID']) - set(ga_prim_runoff_er['UNIQUE_ID'])))
# Create Data Frame
name_anomalies = pd.DataFrame()
name_anomalies['runoff'] = runoff_names
name_anomalies['prim'] = prim_names

In [8]:
#visually inspect
name_anomalies

,runoff,prim
0,Charlton-:-Winokur,Charlton-:-Winkour
1,Chatham-:-1-05C Jacob G. Smith Elementary,Chatham-:-1-05C JEA Building
2,Chatham-:-1-16C Montgomery Athletic Association,Chatham-:-1-16C Ferguson Ave Baptist
3,Chatham-:-7-11C Savannah Fire Station #14,Chatham-:-7-11C Savannah Fire Station 14
4,Fulton-:-11E1,Fulton-:-1.10E+02
5,Fulton-:-11E2,Fulton-:-1.10E+03
6,Fulton-:-11E3,Fulton-:-1.10E+04
7,Fulton-:-12E1,Fulton-:-1.20E+02
8,Liberty-:-PAC,Liberty-:-Button Gwinnett


Using [Charlton county website](https://charltoncountyga.us/264/Polling-Locations) to determine right spelling of Winokur.

Using [Liberty County](https://www.libertycountyga.com/DocumentCenter/View/1115/Notice-of-Relocated-Polling-Places-and-Additional-Advance-Voting-Location?bidId=) website to determine Polling Place change to PAC.

Using [Chatham County GIS](https://chathamelections.sagis.org/) to verify polling place change.

In Fulton County, precinct names are erroneously interpreted in the primary.

In [9]:
# create rename dict to rename primary precincts in the above instances
uid_dict = dict(zip(name_anomalies['prim'], name_anomalies['runoff']))

In [10]:
# apply rename dict
ga_prim_er['UNIQUE_ID'] = ga_prim_er['UNIQUE_ID'].replace(uid_dict)

#Adjust precinct name field to account for name changes
ga_prim_er['precinct'] = ga_prim_er['UNIQUE_ID'].apply(lambda x: x.split('-:-')[1])

In [11]:
# check if all unique_ids match up
set(ga_prim_er['UNIQUE_ID']) == set(ga_prim_runoff_er['UNIQUE_ID'])

True

In [12]:
ga_prim_er.head()

,UNIQUE_ID,COUNTYFP,county,precinct,P22USSDWAR,P22USSDJOH,P22USSRWAL,P22USSRBLA,P22USSRSAD,P22USSRCLA,P22USSRKIN,P22USSRMCC,P22GOVDABR,P22GOVRKEM,P22GOVRPER,P22GOVRTAY,P22GOVRDAV,P22GOVRWIL,P22LTGDHAL,P22LTGDBAI,P22LTGDSHA,P22LTGDBTY,P22LTGDALL,P22LTGDJAC,P22LTGDBTO,P22LTGDHAY,P22LTGDMAL,P22LTGRJON,P22LTGRMIL,P22LTGRMCG,P22LTGRSEA,P22ATGDJOR,P22ATGDSMI,P22ATGRCAR,P22ATGRGOR,P22SOSDNGU,P22SOSDDAW,P22SOSDOWE,P22SOSDGRI,P22SOSDEAV,P22SOSRRAF,P22SOSRHIC,P22SOSRISL,P22SOSRHUD,P22AGRDHEM,P22AGRDDUK,P22AGRDSWA,P22AGRRHAR,P22INSDROB,P22INSDBAK,P22INSDWIL,P22INSRKIN,P22INSRWIT,P22INSRCOW,P22LABDBOD,P22LABDHOR,P22LABDJAC,P22LABDSUR,P22LABDDEA,P22LABRTHO,P22LABRCOA,P22LABRBHA,PPSC02DDUR,PPSC02DEDW,PPSC02RECH,PPSC03DEDW,PPSC03DFAR,PPSC03DMOO,PPSC03RJOH,P22SUPDSEA,P22SUPDHOW,P22SUPDMOR,P22SUPDHIT,P22SUPRWOO,P22SUPRBAR,G22SSCNMCM,G22SSCNLAG,G22SSCNCOL,G22SSCNBRI,G22COANBAR,G22COANMCF,G22COANPIP,PCON01RCAR,PCON01DGRI,PCON01DHER,PCON01DMUN,PCON02DBIS,PCON02DOHA,PCON02RHUN,PCON02RWES,PCON02RJOH,PCON02RCHI,PCON02RROB,PCON02RWHI,PCON03RFER,PCON03RCRA,PCON03DALM,PCON04DJOH,PCON04RCHA,PCON04RIVY,PCON05RZIM,PCON05DWIL,PCON05DSTO,PCON05DMAC,PCON06RMCC,PCON06REVA,PCON06RSTA,PCON06RHAN,PCON06RYU,PCON06RHAR,PCON06RGAT,PCON06RVOY,PCON06RSMI,PCON06DCHR,PCON06DWHI,PCON07RCOR,PCON07RGON,PCON07RMCC,PCON07RWES,PCON07RNYG,PCON07DMCB,PCON07DBOU,PCON07DMCL,PCON08DBUT,PCON08RSCO,PCON09RCLY,PCON09RSOU,PCON09RBOG,PCON09RHOW,PCON09RLON,PCON09DFOR,PCON10RCOL,PCON10RJON,PCON10RBAR,PCON10RBRO,PCON10RCUR,PCON10RSIM,PCON10RMCM,PCON10RSWA,PCON10DJOH,PCON10DFOR,PCON10DHAT,PCON10DODU,PCON10DWAL,PCON11DDAZ,PCON11RLOU,PCON12RALL,PCON12DJOH,PCON13RGON,PCON13RPLO,PCON13RHAW,PCON13DSCO,PCON13DBAK,PCON13DDRI,PCON13DFOR,PCON14DFLO,PCON14DDAV,PCON14DMCC,PCON14RGRE,PCON14RSTR,PCON14RCUN,PCON14RHAY,PCON14RLUT,PCON14RSYN,PSU01DJON,PSU01DNIQ,PSU01RWAT,PSU02RYOU,PSU02RYAS,PSU02DMAL,PSU02DSCO,PSU03RHOD,PSU03RJON,PSU03RHAY,PSU04RHIC,PSU05DRAH,PSU05DPOW,PSU06DEST,PSU06DWAK,PSU06RGLA,PSU06RMOO,PSU07RMCK,PSU07RSAN,PSU07DISL,PSU07DMOO,PSU08RGOO,PSU09DMER,PSU10DJON,PSU10DHOL,PSU11RBUR,PSU12DSIM,PSU13RSUM,PSU13RPER,PSU13RWOO,PSU14DMCL,PSU14RHAU,PSU15DHAR,PSU16RHAR,PSU16DDUB,PSU17RSTR,PSU17RMAU,PSU17DMOR,PSU18RKEN,PSU18RVAN,PSU18DBEN,PSU19RTIL,PSU19DMOO,PSU20RWAL,PSU21RBEA,PSU22RDAN,PSU22DJON,PSU23RBUR,PSU24RAND,PSU25DROD,PSU25RWIL,PSU25RJEF,PSU25ROLI,PSU25RSUL,PSU26DLUC,PSU27DBIN,PSU27RDOL,PSU28RBRA,PSU29DWRI,PSU29DMAN,PSU29RROB,PSU30RDUG,PSU31RANA,PSU32DBEN,PSU32RKIR,PSU32RSOH,PSU33DRHE,PSU33DHEM,PSU34RSMI,PSU34DSEA,PSU35DJAM,PSU35DJON,PSU35DGIL,PSU35DGLA,PSU36DORR,PSU36DSMI,PSU36DSTE,PSU36RHUE,PSU37RSET,PSU37RDEL,PSU37RJOH,PSU37DPAR,PSU37DNIC,PSU38DTAT,PSU38DBRA,PSU38DCAR,PSU38DPET,PSU39DHAL,PSU40RMCD,PSU40DHAR,PSU41DJAC,PSU41RJON,PSU42DPAR,PSU43RWIL,PSU43DAND,PSU43DLES,PSU44DDAV,PSU44DFLO,PSU45DJON,PSU45RDIX,PSU46DFER,PSU46RCOW,PSU47RGIN,PSU47RHAR,PSU47RCHA,PSU47DSCO,PSU48RSTI,PSU48RGRI,PSU48DUDD,PSU49DCOO,PSU49RECH,PSU49RGIB,PSU49RSTR,PSU50DWIL,PSU50RHAT,PSU51RGOO,PSU52RHUF,PSU52RLEW,PSU52RKEE,PSU53RMOO,PSU53RHEN,PSU54RPAY,PSU54RKEE,PSU55DBUT,PSU55DODI,PSU56DTHO,PSU56RALB,PSL001RCAM,PSL001RHAR,PSL002RTAR,PSL002RCOL,PSL002RNOB,PSL003RHOR,PSL003RWEL,PSL004RCAR,PSL004RVOY,PSL005RBAR,PSL006RRID,PSL006RCOK,PSL007RRAL,PSL008DKRI,PSL008RGUN,PSL009RWAD,PSL009RTOL,PSL010RAND,PSL011DHOL,PSL011RJAS,PSL012RLUM,PSL012RWAT,PSL013DDEM,PSL013RMAR,PSL013RBAR,PSL014RSCO,PSL015RGAM,PSL016RKEL,PSL016RRIC,PSL017RMOM,PSL017RWOL,PSL017DMAR,PSL018RSMI,PSL018DRHU,PSL019DCOY,PSL019RGUL,PSL020RBYR,PSL020RSAU,PSL020RHIX,PSL021RTHO,PSL022RRID,PSL022RKOS,PSL022DHIL,PSL023RBAL,PSL023RHUT,PSL024DWAL,PSL024RGIL,PSL024RBAR,PSL024RSOL,PSL025DMEY,PSL025RJON,PSL026RMCD,PSL026DHEL,PSL027RHAW,PSL028RCOX,PSL028RTRE,PSL028RSHO,PSL028RMCC,PSL028RLAN,PSL028RLUC,PSL028DWOO,PSL029RDUB,PSL029DPAN,PSL030DFLO,PSL030RMCC,PSL030RPIM,PSL030RSAN,PSL031RDUN,PSL031RCLE,PSL032RERW,PSL033RPOW,PSL033RPUR,PSL034DCOK,PSL034RSEA,PSL035DCAM,PSL035DRIN,PSL035DMIL,PSL035RTRI,PSL036DRYN,PSL036REHR,PSL037DWIL,PSL037RRED,PSL038DWIL,PSL039DDEL,PSL039DCUM,

In [13]:
# join election results
# merge
ga_er_join = ga_prim_er.merge(ga_prim_runoff_er, on='UNIQUE_ID', how='left', indicator=True)

In [14]:
#check indicator to see if merge was successful
ga_er_join._merge.value_counts()

both          2707
left_only        0
right_only       0
Name: _merge, dtype: int64

In [15]:
ga_er_join.head()

,UNIQUE_ID,COUNTYFP_x,county_x,precinct_x,P22USSDWAR,P22USSDJOH,P22USSRWAL,P22USSRBLA,P22USSRSAD,P22USSRCLA,P22USSRKIN,P22USSRMCC,P22GOVDABR,P22GOVRKEM,P22GOVRPER,P22GOVRTAY,P22GOVRDAV,P22GOVRWIL,P22LTGDHAL,P22LTGDBAI,P22LTGDSHA,P22LTGDBTY,P22LTGDALL,P22LTGDJAC,P22LTGDBTO,P22LTGDHAY,P22LTGDMAL,P22LTGRJON,P22LTGRMIL,P22LTGRMCG,P22LTGRSEA,P22ATGDJOR,P22ATGDSMI,P22ATGRCAR,P22ATGRGOR,P22SOSDNGU,P22SOSDDAW,P22SOSDOWE,P22SOSDGRI,P22SOSDEAV,P22SOSRRAF,P22SOSRHIC,P22SOSRISL,P22SOSRHUD,P22AGRDHEM,P22AGRDDUK,P22AGRDSWA,P22AGRRHAR,P22INSDROB,P22INSDBAK,P22INSDWIL,P22INSRKIN,P22INSRWIT,P22INSRCOW,P22LABDBOD,P22LABDHOR,P22LABDJAC,P22LABDSUR,P22LABDDEA,P22LABRTHO,P22LABRCOA,P22LABRBHA,PPSC02DDUR,PPSC02DEDW,PPSC02RECH,PPSC03DEDW,PPSC03DFAR,PPSC03DMOO,PPSC03RJOH,P22SUPDSEA,P22SUPDHOW,P22SUPDMOR,P22SUPDHIT,P22SUPRWOO,P22SUPRBAR,G22SSCNMCM,G22SSCNLAG,G22SSCNCOL,G22SSCNBRI,G22COANBAR,G22COANMCF,G22COANPIP,PCON01RCAR,PCON01DGRI,PCON01DHER,PCON01DMUN,PCON02DBIS,PCON02DOHA,PCON02RHUN,PCON02RWES,PCON02RJOH,PCON02RCHI,PCON02RROB,PCON02RWHI,PCON03RFER,PCON03RCRA,PCON03DALM,PCON04DJOH,PCON04RCHA,PCON04RIVY,PCON05RZIM,PCON05DWIL,PCON05DSTO,PCON05DMAC,PCON06RMCC,PCON06REVA,PCON06RSTA,PCON06RHAN,PCON06RYU,PCON06RHAR,PCON06RGAT,PCON06RVOY,PCON06RSMI,PCON06DCHR,PCON06DWHI,PCON07RCOR,PCON07RGON,PCON07RMCC,PCON07RWES,PCON07RNYG,PCON07DMCB,PCON07DBOU,PCON07DMCL,PCON08DBUT,PCON08RSCO,PCON09RCLY,PCON09RSOU,PCON09RBOG,PCON09RHOW,PCON09RLON,PCON09DFOR,PCON10RCOL,PCON10RJON,PCON10RBAR,PCON10RBRO,PCON10RCUR,PCON10RSIM,PCON10RMCM,PCON10RSWA,PCON10DJOH,PCON10DFOR,PCON10DHAT,PCON10DODU,PCON10DWAL,PCON11DDAZ,PCON11RLOU,PCON12RALL,PCON12DJOH,PCON13RGON,PCON13RPLO,PCON13RHAW,PCON13DSCO,PCON13DBAK,PCON13DDRI,PCON13DFOR,PCON14DFLO,PCON14DDAV,PCON14DMCC,PCON14RGRE,PCON14RSTR,PCON14RCUN,PCON14RHAY,PCON14RLUT,PCON14RSYN,PSU01DJON,PSU01DNIQ,PSU01RWAT,PSU02RYOU,PSU02RYAS,PSU02DMAL,PSU02DSCO,PSU03RHOD,PSU03RJON,PSU03RHAY,PSU04RHIC,PSU05DRAH,PSU05DPOW,PSU06DEST,PSU06DWAK,PSU06RGLA,PSU06RMOO,PSU07RMCK,PSU07RSAN,PSU07DISL,PSU07DMOO,PSU08RGOO,PSU09DMER,PSU10DJON,PSU10DHOL,PSU11RBUR,PSU12DSIM,PSU13RSUM,PSU13RPER,PSU13RWOO,PSU14DMCL,PSU14RHAU,PSU15DHAR,PSU16RHAR,PSU16DDUB,PSU17RSTR,PSU17RMAU,PSU17DMOR,PSU18RKEN,PSU18RVAN,PSU18DBEN,PSU19RTIL,PSU19DMOO,PSU20RWAL,PSU21RBEA,PSU22RDAN,PSU22DJON,PSU23RBUR,PSU24RAND,PSU25DROD,PSU25RWIL,PSU25RJEF,PSU25ROLI,PSU25RSUL,PSU26DLUC,PSU27DBIN,PSU27RDOL,PSU28RBRA,PSU29DWRI,PSU29DMAN,PSU29RROB,PSU30RDUG,PSU31RANA,PSU32DBEN,PSU32RKIR,PSU32RSOH,PSU33DRHE,PSU33DHEM,PSU34RSMI,PSU34DSEA,PSU35DJAM,PSU35DJON,PSU35DGIL,PSU35DGLA,PSU36DORR,PSU36DSMI,PSU36DSTE,PSU36RHUE,PSU37RSET,PSU37RDEL,PSU37RJOH,PSU37DPAR,PSU37DNIC,PSU38DTAT,PSU38DBRA,PSU38DCAR,PSU38DPET,PSU39DHAL,PSU40RMCD,PSU40DHAR,PSU41DJAC,PSU41RJON,PSU42DPAR,PSU43RWIL,PSU43DAND,PSU43DLES,PSU44DDAV,PSU44DFLO,PSU45DJON,PSU45RDIX,PSU46DFER,PSU46RCOW,PSU47RGIN,PSU47RHAR,PSU47RCHA,PSU47DSCO,PSU48RSTI,PSU48RGRI,PSU48DUDD,PSU49DCOO,PSU49RECH,PSU49RGIB,PSU49RSTR,PSU50DWIL,PSU50RHAT,PSU51RGOO,PSU52RHUF,PSU52RLEW,PSU52RKEE,PSU53RMOO,PSU53RHEN,PSU54RPAY,PSU54RKEE,PSU55DBUT,PSU55DODI,PSU56DTHO,PSU56RALB,PSL001RCAM,PSL001RHAR,PSL002RTAR,PSL002RCOL,PSL002RNOB,PSL003RHOR,PSL003RWEL,PSL004RCAR,PSL004RVOY,PSL005RBAR,PSL006RRID,PSL006RCOK,PSL007RRAL,PSL008DKRI,PSL008RGUN,PSL009RWAD,PSL009RTOL,PSL010RAND,PSL011DHOL,PSL011RJAS,PSL012RLUM,PSL012RWAT,PSL013DDEM,PSL013RMAR,PSL013RBAR,PSL014RSCO,PSL015RGAM,PSL016RKEL,PSL016RRIC,PSL017RMOM,PSL017RWOL,PSL017DMAR,PSL018RSMI,PSL018DRHU,PSL019DCOY,PSL019RGUL,PSL020RBYR,PSL020RSAU,PSL020RHIX,PSL021RTHO,PSL022RRID,PSL022RKOS,PSL022DHIL,PSL023RBAL,PSL023RHUT,PSL024DWAL,PSL024RGIL,PSL024RBAR,PSL024RSOL,PSL025DMEY,PSL025RJON,PSL026RMCD,PSL026DHEL,PSL027RHAW,PSL028RCOX,PSL028RTRE,PSL028RSHO,PSL028RMCC,PSL028RLAN,PSL028RLUC,PSL028DWOO,PSL029RDUB,PSL029DPAN,PSL030DFLO,PSL030RMCC,PSL030RPIM,PSL030RSAN,PSL031RDUN,PSL031RCLE,PSL032RERW,PSL033RPOW,PSL033RPUR,PSL034DCOK,PSL034RSEA,PSL035DCAM,PSL035DRIN,PSL035DMIL,PSL035RTRI,PSL036DRYN,PSL036REHR,PSL037DWIL,PSL037RRED,PSL038DWIL,PSL039DDEL,PSL03

In [16]:
#Drop merge column
ga_er_join.drop(labels = ['_merge', 'COUNTYFP_y','county_y', 'precinct_y'], axis = 1, inplace = True)
#rename columns
ga_er_join.rename(columns={'COUNTYFP_x': 'COUNTYFP', 'county_x': 'county', 'precinct_x':'precinct'}, inplace=True)

### Modify Election Results

In Lowndes county, 'Wiregrass College' precinct voters were added to the 'Rainwater Conference Center' precinct.

In [17]:
#create df with rows to be combined
lowndes_anom = ga_er_join[ga_er_join['UNIQUE_ID'].isin(['Lowndes-:-Wiregrass College', 'Lowndes-:-Rainwater Conference Center'])]

In [18]:
#add totals of vote columns
lowndes_votes = lowndes_anom.iloc[:, 4:].sum(axis=0).to_frame().T
# #identifying columns
# lowndes_details = lowndes_anom.iloc[0, : 4].to_frame().T
lowndes_votes['UNIQUE_ID'] = 'Lowndes-:-Rainwater Conference Center'
lowndes_votes['COUNTYFP'] = '185'
lowndes_votes['county'] = 'Lowndes'
lowndes_votes['precinct'] = 'Rainwater Conference Center'
#Reorder df
lowndes_combined = lowndes_votes[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + lowndes_votes.columns[:-4].to_list()]

In [19]:
# Drop two lowndes precinct rows from election returns
ga_er_lowndes_mod = ga_er_join[~ga_er_join['UNIQUE_ID'].isin(['Lowndes-:-Wiregrass College', 'Lowndes-:-Rainwater Conference Center'])]

In [20]:
# Append modified lowndes to ER
ga_er_mod = ga_er_lowndes_mod.append(lowndes_combined)

In [21]:
#Visually inspect
#ga_er_mod[ga_er_mod['county'] == 'Lowndes']

In [22]:
# Vote Totals check
county_totals_check(ga_er_join, "RDH raw", ga_er_mod, "mod er", ga_er_join.columns[4:].tolist(), "COUNTYFP")

***Countywide Totals Check***

P22USSDWAR is equal across all counties
P22USSDJOH is equal across all counties
P22USSRWAL is equal across all counties
P22USSRBLA is equal across all counties
P22USSRSAD is equal across all counties
P22USSRCLA is equal across all counties
P22USSRKIN is equal across all counties
P22USSRMCC is equal across all counties
P22GOVDABR is equal across all counties
P22GOVRKEM is equal across all counties
P22GOVRPER is equal across all counties
P22GOVRTAY is equal across all counties
P22GOVRDAV is equal across all counties
P22GOVRWIL is equal across all counties
P22LTGDHAL is equal across all counties
P22LTGDBAI is equal across all counties
P22LTGDSHA is equal across all counties
P22LTGDBTY is equal across all counties
P22LTGDALL is equal across all counties
P22LTGDJAC is equal across all counties
P22LTGDBTO is equal across all counties
P22LTGDHAY is equal across all counties
P22LTGDMAL is equal across all counties
P22LTGRJON is equal across all counties
P22LTGRMI

PSU23RBUR is equal across all counties
PSU24RAND is equal across all counties
PSU25DROD is equal across all counties
PSU25RWIL is equal across all counties
PSU25RJEF is equal across all counties
PSU25ROLI is equal across all counties
PSU25RSUL is equal across all counties
PSU26DLUC is equal across all counties
PSU27DBIN is equal across all counties
PSU27RDOL is equal across all counties
PSU28RBRA is equal across all counties
PSU29DWRI is equal across all counties
PSU29DMAN is equal across all counties
PSU29RROB is equal across all counties
PSU30RDUG is equal across all counties
PSU31RANA is equal across all counties
PSU32DBEN is equal across all counties
PSU32RKIR is equal across all counties
PSU32RSOH is equal across all counties
PSU33DRHE is equal across all counties
PSU33DHEM is equal across all counties
PSU34RSMI is equal across all counties
PSU34DSEA is equal across all counties
PSU35DJAM is equal across all counties
PSU35DJON is equal across all counties
PSU35DGIL is equal across

PSL056DJON is equal across all counties
PSL056DCHA is equal across all counties
PSL057DEVA is equal across all counties
PSL058DCAN is equal across all counties
PSL058DTON is equal across all counties
PSL059DOLA is equal across all counties
PSL059DCOL is equal across all counties
PSL060DJON is equal across all counties
PSL060DLEE is equal across all counties
PSL061DBRU is equal across all counties
PSL061DKEM is equal across all counties
PSL061DDAW is equal across all counties
PSL061DMCC is equal across all counties
PSL062DMIL is equal across all counties
PSL062DCAL is equal across all counties
PSL062DNOB is equal across all counties
PSL063DSCH is equal across all counties
PSL064DEDW is equal across all counties
PSL064DTHO is equal across all counties
PSL064DWIL is equal across all counties
PSL064RNEW is equal across all counties
PSL064RMIL is equal across all counties
PSL064RPAR is equal across all counties
PSL065DTHO is equal across all counties
PSL065RHOR is equal across all counties


PSL161RHIT is equal across all counties
PSL161DBAR is equal across all counties
PSL162DGIL is equal across all counties
PSL162DPRA is equal across all counties
PSL163DWES is equal across all counties
PSL163DBOW is equal across all counties
PSL164DTHO is equal across all counties
PSL164RSTE is equal across all counties
PSL164RPAW is equal across all counties
PSL165DJAC is equal across all counties
PSL166RPET is equal across all counties
PSL167RDEL is equal across all counties
PSL168DWIL is equal across all counties
PSL168DSMI is equal across all counties
PSL169RPIR is equal across all counties
PSL169DBRO is equal across all counties
PSL170RHOU is equal across all counties
PSL171RCAM is equal across all counties
PSL172RWAT is equal across all counties
PSL173DJEN is equal across all counties
PSL173RTAY is equal across all counties
PSL174RCOR is equal across all counties
PSL175RLAH is equal across all counties
PSL175RBLA is equal across all counties
PSL176RBUR is equal across all counties


<p><a name="shp"></a></p>

### Modify Precinct Boundaries

#### Paulding County Edits

Using Paulding County base shapefile from GA 2020 General Election Returns

In [25]:
ga_gen_20_shp = ga_gen_20_shp[['UNIQUE_ID', 'COUNTYFP', 'PRECINCT_N', 'CTYNAME', 'geometry']]
#subset to Paulding County, and columns of interest
paulding_shp = ga_gen_20_shp[ga_gen_20_shp['CTYNAME'] == 'PAULDING']

In [26]:
# Rename Paulding 2020 Precincts to match 2022 Primary Election
paulding_rename_df = pd.read_csv('./raw-from-source/ga_name_anoms - paulding_rename.csv')

In [27]:
#Change precinct names in shp to correctly match election returns. 
paulding_dict = dict(zip(paulding_rename_df['shp'], paulding_rename_df['er']))
#apply rename dictionary
paulding_shp['UNIQUE_ID'] = paulding_shp['UNIQUE_ID'].replace(paulding_dict)

In [28]:
# Check 
set(ga_er_mod[ga_er_mod['county'] == 'Paulding']['UNIQUE_ID']) == set(paulding_shp['UNIQUE_ID'])

True

Add Updated Paulding SHP to Basefile

In [29]:
# Drop Paulding precincts from GA 2022 General base shapefile
ga_gen_minus_paulding = ga_gen_shp[ga_gen_shp['county'] != 'Paulding']

In [30]:
#Check - looking for 18
ga_gen_shp.shape[0] - ga_gen_minus_paulding.shape[0]

18

In [31]:
# Append Paulding precinct shapes to 2022 General Basefile
ga_prim_shp = ga_gen_minus_paulding[['UNIQUE_ID', 'geometry']].append(paulding_shp[['UNIQUE_ID', 'geometry']])

#### Chatham County Edits
In the base shapefile, 8 Chatham county precincts are split into two shapes apeice by municipal boundaries for Savannah. These shape pairs are merged into one precinct each to match primary and primary runoff election results.

The shapefiles for two precincts - 2-06 Eli Whitney Complex and 3-15C Eli Whitney Complex use GA 2022 General election shapes.

In [32]:
# # Rename 8 Chatham County Precincts to match 2022 Primary Election
# chatham_rename_df = pd.read_csv('./raw-from-source/chathamshp_rename.csv')
# #Change precinct names in shp to correctly match election returns. 
# chath_dict = dict(zip(chatham_rename_df['SHP'], chatham_rename_df['ER']))
# #apply rename dictionary
# ga_prim_shp['UNIQUE_ID'] = ga_prim_shp['UNIQUE_ID'].replace(chath_dict)

In [33]:
# Drop Eli Whitney precincts from GA 2022 General base shapefile
ga_prim_shp_minus_eli = ga_prim_shp[ga_prim_shp['UNIQUE_ID'] != 'Chatham-:-2-06C Eli Whitney Complex']
# Check
len(ga_prim_shp) - len(ga_prim_shp_minus_eli)

1

In [34]:
#isolate eli whitney complex precinct shapes from GA 2020 General shapefiles
eli_whitney_2020_shp = ga_gen_20_shp[ga_gen_20_shp['UNIQUE_ID'].isin(['051-ELI WHITNEY COMPLEX 3-15C', '051-ELI WHITNEY COMPLEX 2-06C'])]

In [35]:
#match unique ID naming convention
eli_whitney_2020_shp['UNIQUE_ID'] = eli_whitney_2020_shp['UNIQUE_ID'].replace({'051-ELI WHITNEY COMPLEX 3-15C': '3-15C Eli Whitney Complex', '051-ELI WHITNEY COMPLEX 2-06C': '2-06C Eli Whitney Complex'})
eli_whitney_2020_shp['UNIQUE_ID'] = 'Chatham-:-' + eli_whitney_2020_shp['UNIQUE_ID']
#Check
eli_whitney_2020_shp

,UNIQUE_ID,COUNTYFP,PRECINCT_N,CTYNAME,geometry
2259,Chatham-:-3-15C Eli Whitney Complex,051,ELI WHITNEY COMPLEX,CHATHAM,"POLYGON ((-81.02948 32.06200, -81.03000 32.060..."
2261,Chatham-:-2-06C Eli Whitney Complex,051,ELI WHITNEY COMPLEX,CHATHAM,"POLYGON ((-81.05333 32.07291, -81.05336 32.072..."


In [36]:
# Append two precinct shapes to 2022 primary shapefile
ga_prim_shp = ga_prim_shp_minus_eli[['UNIQUE_ID', 'geometry']].append(eli_whitney_2020_shp[['UNIQUE_ID', 'geometry']])

In [37]:
len(ga_prim_shp)

2734

#### Clean shapefile to match precinct names in Election returns

In [38]:
#Check number of rows/precincts
print('Number of precincts in GA Prim SHP:', ga_prim_shp['UNIQUE_ID'].nunique())
print('Number of precincts in GA Prim ER:', ga_er_mod['UNIQUE_ID'].nunique())

Number of precincts in GA Prim SHP: 2734
Number of precincts in GA Prim ER: 2706


In [39]:
rename_df = pd.read_csv('./raw-from-source/ga_name_anoms - shp_rename.csv')

In [40]:
#Change precinct names in shp to correctly match election returns. 
prec_dict = dict(zip(rename_df['shp'], rename_df['er']))

In [41]:
ga_prim_shp['UNIQUE_ID'] = ga_prim_shp['UNIQUE_ID'].replace(prec_dict)

In [42]:
#check before changing df
len(ga_prim_shp.dissolve(by= 'UNIQUE_ID'))

2719

In [43]:
ga_prim_shp_merged = ga_prim_shp.dissolve(by= 'UNIQUE_ID').reset_index()

In [44]:
ga_prim_shp_merged.head()

,UNIQUE_ID,geometry
0,Appling-:-1B,"POLYGON ((-82.37653 31.83940, -82.37665 31.839..."
1,Appling-:-1C,"POLYGON ((-82.49759 31.81825, -82.49872 31.818..."
2,Appling-:-2,"POLYGON ((-82.35743 31.78134, -82.35859 31.781..."
3,Appling-:-3A1,"POLYGON ((-82.19971 31.83475, -82.20023 31.834..."
4,Appling-:-3C,"POLYGON ((-82.30075 31.67965, -82.30342 31.683..."


In [45]:
# Merge two no vote precincts in Fulton County in Election Returns - some single votes were added in via vote allocation function during processing of election returns.
fult = ga_er_mod[ga_er_mod['UNIQUE_ID'].isin(['Fulton-:-CP053', 'Fulton-:-CP052'])]
fult_mod = fult.sum().to_frame().T
fult_mod['UNIQUE_ID'] = 'Fulton-:-CP052'
fult_mod['COUNTYFP'] = 121
fult_mod['county'] = 'Fulton'
fult_mod['precinct'] = 'CP052'
fult_mod

,UNIQUE_ID,COUNTYFP,county,precinct,P22USSDWAR,P22USSDJOH,P22USSRWAL,P22USSRBLA,P22USSRSAD,P22USSRCLA,P22USSRKIN,P22USSRMCC,P22GOVDABR,P22GOVRKEM,P22GOVRPER,P22GOVRTAY,P22GOVRDAV,P22GOVRWIL,P22LTGDHAL,P22LTGDBAI,P22LTGDSHA,P22LTGDBTY,P22LTGDALL,P22LTGDJAC,P22LTGDBTO,P22LTGDHAY,P22LTGDMAL,P22LTGRJON,P22LTGRMIL,P22LTGRMCG,P22LTGRSEA,P22ATGDJOR,P22ATGDSMI,P22ATGRCAR,P22ATGRGOR,P22SOSDNGU,P22SOSDDAW,P22SOSDOWE,P22SOSDGRI,P22SOSDEAV,P22SOSRRAF,P22SOSRHIC,P22SOSRISL,P22SOSRHUD,P22AGRDHEM,P22AGRDDUK,P22AGRDSWA,P22AGRRHAR,P22INSDROB,P22INSDBAK,P22INSDWIL,P22INSRKIN,P22INSRWIT,P22INSRCOW,P22LABDBOD,P22LABDHOR,P22LABDJAC,P22LABDSUR,P22LABDDEA,P22LABRTHO,P22LABRCOA,P22LABRBHA,PPSC02DDUR,PPSC02DEDW,PPSC02RECH,PPSC03DEDW,PPSC03DFAR,PPSC03DMOO,PPSC03RJOH,P22SUPDSEA,P22SUPDHOW,P22SUPDMOR,P22SUPDHIT,P22SUPRWOO,P22SUPRBAR,G22SSCNMCM,G22SSCNLAG,G22SSCNCOL,G22SSCNBRI,G22COANBAR,G22COANMCF,G22COANPIP,PCON01RCAR,PCON01DGRI,PCON01DHER,PCON01DMUN,PCON02DBIS,PCON02DOHA,PCON02RHUN,PCON02RWES,PCON02RJOH,PCON02RCHI,PCON02RROB,PCON02RWHI,PCON03RFER,PCON03RCRA,PCON03DALM,PCON04DJOH,PCON04RCHA,PCON04RIVY,PCON05RZIM,PCON05DWIL,PCON05DSTO,PCON05DMAC,PCON06RMCC,PCON06REVA,PCON06RSTA,PCON06RHAN,PCON06RYU,PCON06RHAR,PCON06RGAT,PCON06RVOY,PCON06RSMI,PCON06DCHR,PCON06DWHI,PCON07RCOR,PCON07RGON,PCON07RMCC,PCON07RWES,PCON07RNYG,PCON07DMCB,PCON07DBOU,PCON07DMCL,PCON08DBUT,PCON08RSCO,PCON09RCLY,PCON09RSOU,PCON09RBOG,PCON09RHOW,PCON09RLON,PCON09DFOR,PCON10RCOL,PCON10RJON,PCON10RBAR,PCON10RBRO,PCON10RCUR,PCON10RSIM,PCON10RMCM,PCON10RSWA,PCON10DJOH,PCON10DFOR,PCON10DHAT,PCON10DODU,PCON10DWAL,PCON11DDAZ,PCON11RLOU,PCON12RALL,PCON12DJOH,PCON13RGON,PCON13RPLO,PCON13RHAW,PCON13DSCO,PCON13DBAK,PCON13DDRI,PCON13DFOR,PCON14DFLO,PCON14DDAV,PCON14DMCC,PCON14RGRE,PCON14RSTR,PCON14RCUN,PCON14RHAY,PCON14RLUT,PCON14RSYN,PSU01DJON,PSU01DNIQ,PSU01RWAT,PSU02RYOU,PSU02RYAS,PSU02DMAL,PSU02DSCO,PSU03RHOD,PSU03RJON,PSU03RHAY,PSU04RHIC,PSU05DRAH,PSU05DPOW,PSU06DEST,PSU06DWAK,PSU06RGLA,PSU06RMOO,PSU07RMCK,PSU07RSAN,PSU07DISL,PSU07DMOO,PSU08RGOO,PSU09DMER,PSU10DJON,PSU10DHOL,PSU11RBUR,PSU12DSIM,PSU13RSUM,PSU13RPER,PSU13RWOO,PSU14DMCL,PSU14RHAU,PSU15DHAR,PSU16RHAR,PSU16DDUB,PSU17RSTR,PSU17RMAU,PSU17DMOR,PSU18RKEN,PSU18RVAN,PSU18DBEN,PSU19RTIL,PSU19DMOO,PSU20RWAL,PSU21RBEA,PSU22RDAN,PSU22DJON,PSU23RBUR,PSU24RAND,PSU25DROD,PSU25RWIL,PSU25RJEF,PSU25ROLI,PSU25RSUL,PSU26DLUC,PSU27DBIN,PSU27RDOL,PSU28RBRA,PSU29DWRI,PSU29DMAN,PSU29RROB,PSU30RDUG,PSU31RANA,PSU32DBEN,PSU32RKIR,PSU32RSOH,PSU33DRHE,PSU33DHEM,PSU34RSMI,PSU34DSEA,PSU35DJAM,PSU35DJON,PSU35DGIL,PSU35DGLA,PSU36DORR,PSU36DSMI,PSU36DSTE,PSU36RHUE,PSU37RSET,PSU37RDEL,PSU37RJOH,PSU37DPAR,PSU37DNIC,PSU38DTAT,PSU38DBRA,PSU38DCAR,PSU38DPET,PSU39DHAL,PSU40RMCD,PSU40DHAR,PSU41DJAC,PSU41RJON,PSU42DPAR,PSU43RWIL,PSU43DAND,PSU43DLES,PSU44DDAV,PSU44DFLO,PSU45DJON,PSU45RDIX,PSU46DFER,PSU46RCOW,PSU47RGIN,PSU47RHAR,PSU47RCHA,PSU47DSCO,PSU48RSTI,PSU48RGRI,PSU48DUDD,PSU49DCOO,PSU49RECH,PSU49RGIB,PSU49RSTR,PSU50DWIL,PSU50RHAT,PSU51RGOO,PSU52RHUF,PSU52RLEW,PSU52RKEE,PSU53RMOO,PSU53RHEN,PSU54RPAY,PSU54RKEE,PSU55DBUT,PSU55DODI,PSU56DTHO,PSU56RALB,PSL001RCAM,PSL001RHAR,PSL002RTAR,PSL002RCOL,PSL002RNOB,PSL003RHOR,PSL003RWEL,PSL004RCAR,PSL004RVOY,PSL005RBAR,PSL006RRID,PSL006RCOK,PSL007RRAL,PSL008DKRI,PSL008RGUN,PSL009RWAD,PSL009RTOL,PSL010RAND,PSL011DHOL,PSL011RJAS,PSL012RLUM,PSL012RWAT,PSL013DDEM,PSL013RMAR,PSL013RBAR,PSL014RSCO,PSL015RGAM,PSL016RKEL,PSL016RRIC,PSL017RMOM,PSL017RWOL,PSL017DMAR,PSL018RSMI,PSL018DRHU,PSL019DCOY,PSL019RGUL,PSL020RBYR,PSL020RSAU,PSL020RHIX,PSL021RTHO,PSL022RRID,PSL022RKOS,PSL022DHIL,PSL023RBAL,PSL023RHUT,PSL024DWAL,PSL024RGIL,PSL024RBAR,PSL024RSOL,PSL025DMEY,PSL025RJON,PSL026RMCD,PSL026DHEL,PSL027RHAW,PSL028RCOX,PSL028RTRE,PSL028RSHO,PSL028RMCC,PSL028RLAN,PSL028RLUC,PSL028DWOO,PSL029RDUB,PSL029DPAN,PSL030DFLO,PSL030RMCC,PSL030RPIM,PSL030RSAN,PSL031RDUN,PSL031RCLE,PSL032RERW,PSL033RPOW,PSL033RPUR,PSL034DCOK,PSL034RSEA,PSL035DCAM,PSL035DRIN,PSL035DMIL,PSL035RTRI,PSL036DRYN,PSL036REHR,PSL037DWIL,PSL037RRED,PSL038DWIL,PSL039DDEL,PSL039DCUM,

In [46]:
fult

,UNIQUE_ID,COUNTYFP,county,precinct,P22USSDWAR,P22USSDJOH,P22USSRWAL,P22USSRBLA,P22USSRSAD,P22USSRCLA,P22USSRKIN,P22USSRMCC,P22GOVDABR,P22GOVRKEM,P22GOVRPER,P22GOVRTAY,P22GOVRDAV,P22GOVRWIL,P22LTGDHAL,P22LTGDBAI,P22LTGDSHA,P22LTGDBTY,P22LTGDALL,P22LTGDJAC,P22LTGDBTO,P22LTGDHAY,P22LTGDMAL,P22LTGRJON,P22LTGRMIL,P22LTGRMCG,P22LTGRSEA,P22ATGDJOR,P22ATGDSMI,P22ATGRCAR,P22ATGRGOR,P22SOSDNGU,P22SOSDDAW,P22SOSDOWE,P22SOSDGRI,P22SOSDEAV,P22SOSRRAF,P22SOSRHIC,P22SOSRISL,P22SOSRHUD,P22AGRDHEM,P22AGRDDUK,P22AGRDSWA,P22AGRRHAR,P22INSDROB,P22INSDBAK,P22INSDWIL,P22INSRKIN,P22INSRWIT,P22INSRCOW,P22LABDBOD,P22LABDHOR,P22LABDJAC,P22LABDSUR,P22LABDDEA,P22LABRTHO,P22LABRCOA,P22LABRBHA,PPSC02DDUR,PPSC02DEDW,PPSC02RECH,PPSC03DEDW,PPSC03DFAR,PPSC03DMOO,PPSC03RJOH,P22SUPDSEA,P22SUPDHOW,P22SUPDMOR,P22SUPDHIT,P22SUPRWOO,P22SUPRBAR,G22SSCNMCM,G22SSCNLAG,G22SSCNCOL,G22SSCNBRI,G22COANBAR,G22COANMCF,G22COANPIP,PCON01RCAR,PCON01DGRI,PCON01DHER,PCON01DMUN,PCON02DBIS,PCON02DOHA,PCON02RHUN,PCON02RWES,PCON02RJOH,PCON02RCHI,PCON02RROB,PCON02RWHI,PCON03RFER,PCON03RCRA,PCON03DALM,PCON04DJOH,PCON04RCHA,PCON04RIVY,PCON05RZIM,PCON05DWIL,PCON05DSTO,PCON05DMAC,PCON06RMCC,PCON06REVA,PCON06RSTA,PCON06RHAN,PCON06RYU,PCON06RHAR,PCON06RGAT,PCON06RVOY,PCON06RSMI,PCON06DCHR,PCON06DWHI,PCON07RCOR,PCON07RGON,PCON07RMCC,PCON07RWES,PCON07RNYG,PCON07DMCB,PCON07DBOU,PCON07DMCL,PCON08DBUT,PCON08RSCO,PCON09RCLY,PCON09RSOU,PCON09RBOG,PCON09RHOW,PCON09RLON,PCON09DFOR,PCON10RCOL,PCON10RJON,PCON10RBAR,PCON10RBRO,PCON10RCUR,PCON10RSIM,PCON10RMCM,PCON10RSWA,PCON10DJOH,PCON10DFOR,PCON10DHAT,PCON10DODU,PCON10DWAL,PCON11DDAZ,PCON11RLOU,PCON12RALL,PCON12DJOH,PCON13RGON,PCON13RPLO,PCON13RHAW,PCON13DSCO,PCON13DBAK,PCON13DDRI,PCON13DFOR,PCON14DFLO,PCON14DDAV,PCON14DMCC,PCON14RGRE,PCON14RSTR,PCON14RCUN,PCON14RHAY,PCON14RLUT,PCON14RSYN,PSU01DJON,PSU01DNIQ,PSU01RWAT,PSU02RYOU,PSU02RYAS,PSU02DMAL,PSU02DSCO,PSU03RHOD,PSU03RJON,PSU03RHAY,PSU04RHIC,PSU05DRAH,PSU05DPOW,PSU06DEST,PSU06DWAK,PSU06RGLA,PSU06RMOO,PSU07RMCK,PSU07RSAN,PSU07DISL,PSU07DMOO,PSU08RGOO,PSU09DMER,PSU10DJON,PSU10DHOL,PSU11RBUR,PSU12DSIM,PSU13RSUM,PSU13RPER,PSU13RWOO,PSU14DMCL,PSU14RHAU,PSU15DHAR,PSU16RHAR,PSU16DDUB,PSU17RSTR,PSU17RMAU,PSU17DMOR,PSU18RKEN,PSU18RVAN,PSU18DBEN,PSU19RTIL,PSU19DMOO,PSU20RWAL,PSU21RBEA,PSU22RDAN,PSU22DJON,PSU23RBUR,PSU24RAND,PSU25DROD,PSU25RWIL,PSU25RJEF,PSU25ROLI,PSU25RSUL,PSU26DLUC,PSU27DBIN,PSU27RDOL,PSU28RBRA,PSU29DWRI,PSU29DMAN,PSU29RROB,PSU30RDUG,PSU31RANA,PSU32DBEN,PSU32RKIR,PSU32RSOH,PSU33DRHE,PSU33DHEM,PSU34RSMI,PSU34DSEA,PSU35DJAM,PSU35DJON,PSU35DGIL,PSU35DGLA,PSU36DORR,PSU36DSMI,PSU36DSTE,PSU36RHUE,PSU37RSET,PSU37RDEL,PSU37RJOH,PSU37DPAR,PSU37DNIC,PSU38DTAT,PSU38DBRA,PSU38DCAR,PSU38DPET,PSU39DHAL,PSU40RMCD,PSU40DHAR,PSU41DJAC,PSU41RJON,PSU42DPAR,PSU43RWIL,PSU43DAND,PSU43DLES,PSU44DDAV,PSU44DFLO,PSU45DJON,PSU45RDIX,PSU46DFER,PSU46RCOW,PSU47RGIN,PSU47RHAR,PSU47RCHA,PSU47DSCO,PSU48RSTI,PSU48RGRI,PSU48DUDD,PSU49DCOO,PSU49RECH,PSU49RGIB,PSU49RSTR,PSU50DWIL,PSU50RHAT,PSU51RGOO,PSU52RHUF,PSU52RLEW,PSU52RKEE,PSU53RMOO,PSU53RHEN,PSU54RPAY,PSU54RKEE,PSU55DBUT,PSU55DODI,PSU56DTHO,PSU56RALB,PSL001RCAM,PSL001RHAR,PSL002RTAR,PSL002RCOL,PSL002RNOB,PSL003RHOR,PSL003RWEL,PSL004RCAR,PSL004RVOY,PSL005RBAR,PSL006RRID,PSL006RCOK,PSL007RRAL,PSL008DKRI,PSL008RGUN,PSL009RWAD,PSL009RTOL,PSL010RAND,PSL011DHOL,PSL011RJAS,PSL012RLUM,PSL012RWAT,PSL013DDEM,PSL013RMAR,PSL013RBAR,PSL014RSCO,PSL015RGAM,PSL016RKEL,PSL016RRIC,PSL017RMOM,PSL017RWOL,PSL017DMAR,PSL018RSMI,PSL018DRHU,PSL019DCOY,PSL019RGUL,PSL020RBYR,PSL020RSAU,PSL020RHIX,PSL021RTHO,PSL022RRID,PSL022RKOS,PSL022DHIL,PSL023RBAL,PSL023RHUT,PSL024DWAL,PSL024RGIL,PSL024RBAR,PSL024RSOL,PSL025DMEY,PSL025RJON,PSL026RMCD,PSL026DHEL,PSL027RHAW,PSL028RCOX,PSL028RTRE,PSL028RSHO,PSL028RMCC,PSL028RLAN,PSL028RLUC,PSL028DWOO,PSL029RDUB,PSL029DPAN,PSL030DFLO,PSL030RMCC,PSL030RPIM,PSL030RSAN,PSL031RDUN,PSL031RCLE,PSL032RERW,PSL033RPOW,PSL033RPUR,PSL034DCOK,PSL034RSEA,PSL035DCAM,PSL035DRIN,PSL035DMIL,PSL035RTRI,PSL036DRYN,PSL036REHR,PSL037DWIL,PSL037RRED,PSL038DWIL,PSL039DDEL,PSL039DCUM,

In [47]:
non_zero_cols = fult.columns[(fult != 0).any()]
fult[non_zero_cols].to_csv('GA_PRIM22_Fulton_VoteAlloc_Anoms', index = False)

In [48]:
fult[non_zero_cols]

,UNIQUE_ID,COUNTYFP,county,precinct,P22USSDWAR,P22GOVDABR,P22LTGDJAC,P22ATGDSMI,P22SOSDOWE,P22AGRDDUK,P22INSDROB,P22LABDSUR,PPSC02DEDW,PPSC03DMOO,P22SUPDSEA,G22SSCNMCM,G22SSCNLAG,G22SSCNBRI,G22COANBAR,G22COANMCF,G22COANPIP,PCON05DWIL,PSU39DHAL,PSL062DMIL
1389,Fulton-:-CP052,121,Fulton,CP052,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1390,Fulton-:-CP053,121,Fulton,CP053,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [49]:
# Drop original precincts from election returns
ga_er_minus_fult = ga_er_mod[~ga_er_mod['UNIQUE_ID'].isin(['Fulton-:-CP053', 'Fulton-:-CP052'])]
#check
len(ga_er_mod) - len(ga_er_minus_fult)

2

In [50]:
# Append combined precinct shape to election returns
ga_er = ga_er_minus_fult.append(fult_mod)
#check
len(ga_er) - len(ga_er_minus_fult)

1

<p><a name="maup"></a></p>

### Join with Election Results

In [51]:
# check that all unique_ids match up
set(ga_prim_shp_merged['UNIQUE_ID']) == set(ga_er['UNIQUE_ID'])

False

In [52]:
# Look at further name anomalies
shp_names = sorted(list(set(ga_prim_shp_merged['UNIQUE_ID']) - set(ga_er['UNIQUE_ID'])))
er_names = sorted(list(set(ga_er['UNIQUE_ID']) - set(ga_prim_shp_merged['UNIQUE_ID'])))

In [53]:
#visually inspect
#er_names # this list should be empty
shp_names # make sure this list has no named precincts, aka precincts with voters

['Bryan-:-029FTSTEW',
 'Chatham-:-051XFTPU',
 'Chattahoochee-:-053FTBEN1',
 'Chattahoochee-:-053FTBEN2',
 'Chattahoochee-:-053FTBEN3',
 'Fulton-:-08F2',
 'Fulton-:-12E2',
 'Fulton-:-AP01E',
 'Fulton-:-SS29B',
 'Muscogee-:-215FTBEN1',
 'Muscogee-:-215FTBEN2',
 'Muscogee-:-215FTBEN3',
 'Muscogee-:-215FTBEN4',
 'Muscogee-:-215ZZZZZZ']

There are 14 geometries with no votes associated, in the shapefile.

In [54]:
# merge
ga_prim_pber = ga_prim_shp_merged[['UNIQUE_ID', 'geometry']].merge(ga_er, on='UNIQUE_ID', how='outer', indicator=True)

In [55]:
#check indicator to see if merge was successful, looking for 14 left only
ga_prim_pber._merge.value_counts()

both          2705
left_only       14
right_only       0
Name: _merge, dtype: int64

In [56]:
ga_prim_pber.head(2)

,UNIQUE_ID,geometry,COUNTYFP,county,precinct,P22USSDWAR,P22USSDJOH,P22USSRWAL,P22USSRBLA,P22USSRSAD,P22USSRCLA,P22USSRKIN,P22USSRMCC,P22GOVDABR,P22GOVRKEM,P22GOVRPER,P22GOVRTAY,P22GOVRDAV,P22GOVRWIL,P22LTGDHAL,P22LTGDBAI,P22LTGDSHA,P22LTGDBTY,P22LTGDALL,P22LTGDJAC,P22LTGDBTO,P22LTGDHAY,P22LTGDMAL,P22LTGRJON,P22LTGRMIL,P22LTGRMCG,P22LTGRSEA,P22ATGDJOR,P22ATGDSMI,P22ATGRCAR,P22ATGRGOR,P22SOSDNGU,P22SOSDDAW,P22SOSDOWE,P22SOSDGRI,P22SOSDEAV,P22SOSRRAF,P22SOSRHIC,P22SOSRISL,P22SOSRHUD,P22AGRDHEM,P22AGRDDUK,P22AGRDSWA,P22AGRRHAR,P22INSDROB,P22INSDBAK,P22INSDWIL,P22INSRKIN,P22INSRWIT,P22INSRCOW,P22LABDBOD,P22LABDHOR,P22LABDJAC,P22LABDSUR,P22LABDDEA,P22LABRTHO,P22LABRCOA,P22LABRBHA,PPSC02DDUR,PPSC02DEDW,PPSC02RECH,PPSC03DEDW,PPSC03DFAR,PPSC03DMOO,PPSC03RJOH,P22SUPDSEA,P22SUPDHOW,P22SUPDMOR,P22SUPDHIT,P22SUPRWOO,P22SUPRBAR,G22SSCNMCM,G22SSCNLAG,G22SSCNCOL,G22SSCNBRI,G22COANBAR,G22COANMCF,G22COANPIP,PCON01RCAR,PCON01DGRI,PCON01DHER,PCON01DMUN,PCON02DBIS,PCON02DOHA,PCON02RHUN,PCON02RWES,PCON02RJOH,PCON02RCHI,PCON02RROB,PCON02RWHI,PCON03RFER,PCON03RCRA,PCON03DALM,PCON04DJOH,PCON04RCHA,PCON04RIVY,PCON05RZIM,PCON05DWIL,PCON05DSTO,PCON05DMAC,PCON06RMCC,PCON06REVA,PCON06RSTA,PCON06RHAN,PCON06RYU,PCON06RHAR,PCON06RGAT,PCON06RVOY,PCON06RSMI,PCON06DCHR,PCON06DWHI,PCON07RCOR,PCON07RGON,PCON07RMCC,PCON07RWES,PCON07RNYG,PCON07DMCB,PCON07DBOU,PCON07DMCL,PCON08DBUT,PCON08RSCO,PCON09RCLY,PCON09RSOU,PCON09RBOG,PCON09RHOW,PCON09RLON,PCON09DFOR,PCON10RCOL,PCON10RJON,PCON10RBAR,PCON10RBRO,PCON10RCUR,PCON10RSIM,PCON10RMCM,PCON10RSWA,PCON10DJOH,PCON10DFOR,PCON10DHAT,PCON10DODU,PCON10DWAL,PCON11DDAZ,PCON11RLOU,PCON12RALL,PCON12DJOH,PCON13RGON,PCON13RPLO,PCON13RHAW,PCON13DSCO,PCON13DBAK,PCON13DDRI,PCON13DFOR,PCON14DFLO,PCON14DDAV,PCON14DMCC,PCON14RGRE,PCON14RSTR,PCON14RCUN,PCON14RHAY,PCON14RLUT,PCON14RSYN,PSU01DJON,PSU01DNIQ,PSU01RWAT,PSU02RYOU,PSU02RYAS,PSU02DMAL,PSU02DSCO,PSU03RHOD,PSU03RJON,PSU03RHAY,PSU04RHIC,PSU05DRAH,PSU05DPOW,PSU06DEST,PSU06DWAK,PSU06RGLA,PSU06RMOO,PSU07RMCK,PSU07RSAN,PSU07DISL,PSU07DMOO,PSU08RGOO,PSU09DMER,PSU10DJON,PSU10DHOL,PSU11RBUR,PSU12DSIM,PSU13RSUM,PSU13RPER,PSU13RWOO,PSU14DMCL,PSU14RHAU,PSU15DHAR,PSU16RHAR,PSU16DDUB,PSU17RSTR,PSU17RMAU,PSU17DMOR,PSU18RKEN,PSU18RVAN,PSU18DBEN,PSU19RTIL,PSU19DMOO,PSU20RWAL,PSU21RBEA,PSU22RDAN,PSU22DJON,PSU23RBUR,PSU24RAND,PSU25DROD,PSU25RWIL,PSU25RJEF,PSU25ROLI,PSU25RSUL,PSU26DLUC,PSU27DBIN,PSU27RDOL,PSU28RBRA,PSU29DWRI,PSU29DMAN,PSU29RROB,PSU30RDUG,PSU31RANA,PSU32DBEN,PSU32RKIR,PSU32RSOH,PSU33DRHE,PSU33DHEM,PSU34RSMI,PSU34DSEA,PSU35DJAM,PSU35DJON,PSU35DGIL,PSU35DGLA,PSU36DORR,PSU36DSMI,PSU36DSTE,PSU36RHUE,PSU37RSET,PSU37RDEL,PSU37RJOH,PSU37DPAR,PSU37DNIC,PSU38DTAT,PSU38DBRA,PSU38DCAR,PSU38DPET,PSU39DHAL,PSU40RMCD,PSU40DHAR,PSU41DJAC,PSU41RJON,PSU42DPAR,PSU43RWIL,PSU43DAND,PSU43DLES,PSU44DDAV,PSU44DFLO,PSU45DJON,PSU45RDIX,PSU46DFER,PSU46RCOW,PSU47RGIN,PSU47RHAR,PSU47RCHA,PSU47DSCO,PSU48RSTI,PSU48RGRI,PSU48DUDD,PSU49DCOO,PSU49RECH,PSU49RGIB,PSU49RSTR,PSU50DWIL,PSU50RHAT,PSU51RGOO,PSU52RHUF,PSU52RLEW,PSU52RKEE,PSU53RMOO,PSU53RHEN,PSU54RPAY,PSU54RKEE,PSU55DBUT,PSU55DODI,PSU56DTHO,PSU56RALB,PSL001RCAM,PSL001RHAR,PSL002RTAR,PSL002RCOL,PSL002RNOB,PSL003RHOR,PSL003RWEL,PSL004RCAR,PSL004RVOY,PSL005RBAR,PSL006RRID,PSL006RCOK,PSL007RRAL,PSL008DKRI,PSL008RGUN,PSL009RWAD,PSL009RTOL,PSL010RAND,PSL011DHOL,PSL011RJAS,PSL012RLUM,PSL012RWAT,PSL013DDEM,PSL013RMAR,PSL013RBAR,PSL014RSCO,PSL015RGAM,PSL016RKEL,PSL016RRIC,PSL017RMOM,PSL017RWOL,PSL017DMAR,PSL018RSMI,PSL018DRHU,PSL019DCOY,PSL019RGUL,PSL020RBYR,PSL020RSAU,PSL020RHIX,PSL021RTHO,PSL022RRID,PSL022RKOS,PSL022DHIL,PSL023RBAL,PSL023RHUT,PSL024DWAL,PSL024RGIL,PSL024RBAR,PSL024RSOL,PSL025DMEY,PSL025RJON,PSL026RMCD,PSL026DHEL,PSL027RHAW,PSL028RCOX,PSL028RTRE,PSL028RSHO,PSL028RMCC,PSL028RLAN,PSL028RLUC,PSL028DWOO,PSL029RDUB,PSL029DPAN,PSL030DFLO,PSL030RMCC,PSL030RPIM,PSL030RSAN,PSL031RDUN,PSL031RCLE,PSL032RERW,PSL033RPOW,PSL033RPUR,PSL034DCOK,PSL034RSEA,PSL035DCAM,PSL035DRIN,PSL035DMIL,PSL035RTRI,PSL036DRYN,PSL036REHR,PSL037DWIL,PSL037RRED,PSL038DWIL,PSL039DDEL,PS

In [57]:
#Drop Merge Column
ga_prim_pber = ga_prim_pber.drop(columns=['_merge'])

In [58]:
# rearrange columns
ga_prim_pber = ga_prim_pber[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + ga_prim_pber.columns[5:].to_list() + ['geometry']]

In [59]:
ga_prim_pber['geometry'] = ga_prim_pber.geometry.buffer(0)

In [60]:
#Fill NA
ga_prim_pber = ga_prim_pber.fillna(0)

In [61]:
#Add details to 14 geometries without votes
novotes = ga_prim_pber[ga_prim_pber['UNIQUE_ID'].isin(shp_names)]
novotes['precinct'] = novotes['UNIQUE_ID'].apply(lambda x: x.split('-:-')[1])
novotes['county'] = novotes['UNIQUE_ID'].apply(lambda x: x.split('-:-')[0])
novotes['COUNTYFP'] = [29, 51, 53, 53, 53, 121, 121, 121, 121, 215, 215, 215, 215, 215]

In [62]:
#Merge edits
ga_prim_final = ga_prim_pber[~ga_prim_pber['UNIQUE_ID'].isin(shp_names)].append(novotes)

#### Add FIPS code

In [64]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")

In [65]:
fips_file = fips_file[fips_file["State"]=="Georgia"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("De Kalb","DeKalb")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
ga_prim_final['COUNTYFP'] = ga_prim_final['county'].map(fips_dict).fillna(ga_prim_final['county'])
ga_prim_final['COUNTYFP'] = ga_prim_final['COUNTYFP'].astype(str)
ga_prim_final['COUNTYFP'] = ga_prim_final['COUNTYFP'].str.zfill(3)

In [69]:
#Check for 159 counties
ga_prim_final['COUNTYFP'].nunique()

159

<p><a name="splits"></a></p>

### Split Precincts

In [71]:
#Assign columns to datasets
unsplit_col_names = ga_prim_final.columns[4:-1].to_list()
cong_cols = [col for col in unsplit_col_names if col.startswith(('PCON', 'RCON'))]
sldu_cols = [col for col in unsplit_col_names if col.startswith(('PSU', 'RSU'))]
sldl_cols = [col for col in unsplit_col_names if col.startswith(('PSL', 'RSL'))]
st_cols = [col for col in unsplit_col_names if col not in cong_cols+sldu_cols+sldl_cols]

In [72]:
# set columns with votes as integer type
for item in unsplit_col_names:
    ga_prim_final[item] = ga_prim_final[item].astype(int)

In [73]:
#Check
len(unsplit_col_names) == len(cong_cols+sldu_cols+sldl_cols+st_cols)

True

In [74]:
#Create datasets for splits
cong = ga_prim_final[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + cong_cols + ['geometry']]
sldu = ga_prim_final[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + sldu_cols + ['geometry']]
sldl = ga_prim_final[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + sldl_cols + ['geometry']]
st = ga_prim_final[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct'] + st_cols + ['geometry']]

#### Identify split precincts

#### Congressional Districts

In [75]:
def get_level_dist(column_name):
    if column_name[0:3] == "PSL" or column_name[0:3] == "RSL":
        level = "SL"
        dist = column_name[3:5]
    elif column_name[0:3] == "PSU" or column_name[0:3] == "RSU":
        level = "SU"
        dist = column_name[3:5]
    elif column_name[0:4] == "PCON" or column_name[0:4] == "RCON":
        level = "CON"
        dist = column_name[4:6]
    else:
        print(column_name)
        raise ValueError
    return level,dist

In [76]:
races = cong_cols
precinct_mapping_dict = {}
split_precincts_list = {}
for index,row in cong.iterrows():
    precinct_list = []
    for contest in races:
        if(row[contest]!=0):
            precinct_info = get_level_dist(contest)
            if precinct_info not in precinct_list:
                precinct_list.append(get_level_dist(contest))
    is_split = is_split_precinct(precinct_list)
    if (is_split):
        split_precincts_list[row["UNIQUE_ID"]]=is_split
    precinct_mapping_dict[row["UNIQUE_ID"]]=precinct_list
    
#Standardize district format with name different from PBER to use later and distinguish when splits occur
ga_cong['HOUSE_DIST']=ga_cong["DISTRICT"].apply(lambda x: str(x).lstrip("0").zfill(2))
cong["CONG_DIST"] = cong.apply(lambda row: return_cd(row, races), axis = 1)

split_precincts_list

{'Bibb-:-Vineville 6': {'CON': ['02', '08']},
 'Cherokee-:-Hickory Flat': {'CON': ['06', '11']},
 'Cobb-:-Big Shanty 02': {'CON': ['06', '11']},
 'Cobb-:-Clarkdale 02': {'CON': ['13', '14']},
 'Cobb-:-East Piedmont 01': {'CON': ['06', '11']},
 'Cobb-:-Eastside 02': {'CON': ['06', '11']},
 'Cobb-:-Elizabeth 01': {'CON': ['06', '11']},
 'Cobb-:-Fair Oaks 02': {'CON': ['11', '13']},
 'Cobb-:-Kemp 02': {'CON': ['11', '14']},
 'Cobb-:-Lost Mountain 03': {'CON': ['11', '14']},
 'Cobb-:-Mableton 01': {'CON': ['13', '14']},
 'Cobb-:-Mableton 02': {'CON': ['13', '14']},
 'Cobb-:-Marietta 5A': {'CON': ['06', '11']},
 'Cobb-:-Marietta 5B': {'CON': ['06', '11']},
 'Cobb-:-Marietta 6A': {'CON': ['06', '11']},
 'Cobb-:-McEachern 01': {'CON': ['13', '14']},
 'Cobb-:-Mt. Bethel 01': {'CON': ['06', '11']},
 'Cobb-:-Nickajack 01': {'CON': ['11', '13']},
 'Cobb-:-Oakdale 01': {'CON': ['11', '13']},
 'Cobb-:-Oregon 01': {'CON': ['11', '14']},
 'Cobb-:-Oregon 05': {'CON': ['11', '14']},
 'Cobb-:-Palmer 01'

In [77]:
#Create Split GDF
cong_w_splits = cong.copy()
for val in cong["UNIQUE_ID"]:
    sl_list = []
    if val in split_precincts_list.keys():
        cong_w_splits = district_splits_mod(split_precincts_list[val],"CONG",val, cong_w_splits, ga_cong, "UNIQUE_ID", 'HOUSE_DIST', races, "CONG_DIST")

In [78]:
#check
set(cong_w_splits['UNIQUE_ID']) - set(cong['UNIQUE_ID'])

{'Bibb-:-Vineville 6-(CONG-02)',
 'Bibb-:-Vineville 6-(CONG-08)',
 'Cherokee-:-Hickory Flat-(CONG-06)',
 'Cherokee-:-Hickory Flat-(CONG-11)',
 'Cobb-:-Big Shanty 02-(CONG-11)',
 'Cobb-:-Clarkdale 02-(CONG-14)',
 'Cobb-:-East Piedmont 01-(CONG-06)',
 'Cobb-:-East Piedmont 01-(CONG-11)',
 'Cobb-:-Eastside 02-(CONG-06)',
 'Cobb-:-Eastside 02-(CONG-11)',
 'Cobb-:-Elizabeth 01-(CONG-06)',
 'Cobb-:-Elizabeth 01-(CONG-11)',
 'Cobb-:-Fair Oaks 02-(CONG-13)',
 'Cobb-:-Kemp 02-(CONG-11)',
 'Cobb-:-Kemp 02-(CONG-14)',
 'Cobb-:-Lost Mountain 03-(CONG-14)',
 'Cobb-:-Mableton 01-(CONG-13)',
 'Cobb-:-Mableton 01-(CONG-14)',
 'Cobb-:-Mableton 02-(CONG-13)',
 'Cobb-:-Mableton 02-(CONG-14)',
 'Cobb-:-Marietta 5A-(CONG-06)',
 'Cobb-:-Marietta 5A-(CONG-11)',
 'Cobb-:-Marietta 5B-(CONG-06)',
 'Cobb-:-Marietta 5B-(CONG-11)',
 'Cobb-:-Marietta 6A-(CONG-06)',
 'Cobb-:-Marietta 6A-(CONG-11)',
 'Cobb-:-McEachern 01-(CONG-14)',
 'Cobb-:-Mt. Bethel 01-(CONG-06)',
 'Cobb-:-Nickajack 01-(CONG-11)',
 'Cobb-:-Nickaja

In [79]:
#fillna
cong_w_splits = cong_w_splits.fillna(0)
#Adjust precinct name field to account for name changes
cong_w_splits['Precinct'] = cong_w_splits['UNIQUE_ID'].apply(lambda x: x.split('-:-')[1])
#reorder df
cong_w_splits = cong_w_splits[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct', 'CONG_DIST'] + cong_cols + ['geometry']]
#check
cong_w_splits[cong_w_splits["CONG_DIST"].isna()]

,UNIQUE_ID,COUNTYFP,county,precinct,CONG_DIST,PCON01RCAR,PCON01DGRI,PCON01DHER,PCON01DMUN,PCON02DBIS,PCON02DOHA,PCON02RHUN,PCON02RWES,PCON02RJOH,PCON02RCHI,PCON02RROB,PCON02RWHI,PCON03RFER,PCON03RCRA,PCON03DALM,PCON04DJOH,PCON04RCHA,PCON04RIVY,PCON05RZIM,PCON05DWIL,PCON05DSTO,PCON05DMAC,PCON06RMCC,PCON06REVA,PCON06RSTA,PCON06RHAN,PCON06RYU,PCON06RHAR,PCON06RGAT,PCON06RVOY,PCON06RSMI,PCON06DCHR,PCON06DWHI,PCON07RCOR,PCON07RGON,PCON07RMCC,PCON07RWES,PCON07RNYG,PCON07DMCB,PCON07DBOU,PCON07DMCL,PCON08DBUT,PCON08RSCO,PCON09RCLY,PCON09RSOU,PCON09RBOG,PCON09RHOW,PCON09RLON,PCON09DFOR,PCON10RCOL,PCON10RJON,PCON10RBAR,PCON10RBRO,PCON10RCUR,PCON10RSIM,PCON10RMCM,PCON10RSWA,PCON10DJOH,PCON10DFOR,PCON10DHAT,PCON10DODU,PCON10DWAL,PCON11DDAZ,PCON11RLOU,PCON12RALL,PCON12DJOH,PCON13RGON,PCON13RPLO,PCON13RHAW,PCON13DSCO,PCON13DBAK,PCON13DDRI,PCON13DFOR,PCON14DFLO,PCON14DDAV,PCON14DMCC,PCON14RGRE,PCON14RSTR,PCON14RCUN,PCON14RHAY,PCON14RLUT,PCON14RSYN,RCON01DHER,RCON01DGRI,RCON02RWES,RCON02RHUN,RCON06RMCC,RCON06REVA,RCON07RGON,RCON07RCOR,RCON10DJOH,RCON10DFOR,RCON10RCOL,RCON10RJON,geometry


#### State Senate Districts

In [80]:
races = sldu_cols
precinct_mapping_dict = {}
split_precincts_list = {}
for index,row in sldu.iterrows():
    precinct_list = []
    for contest in races:
        if(row[contest]!=0):
            precinct_info = get_level_dist(contest)
            if precinct_info not in precinct_list:
                precinct_list.append(get_level_dist(contest))
    is_split = is_split_precinct(precinct_list)
    if (is_split):
        split_precincts_list[row["UNIQUE_ID"]]=is_split
    precinct_mapping_dict[row["UNIQUE_ID"]]=precinct_list
    
#Standardize district format with name different from PBER to use later and distinguish when splits occur
ga_sldu['SS_DIST']=ga_sldu["DISTRICT"].astype(str).str.zfill(2)
sldu["SLDU_DIST"] = sldu.apply(lambda row: return_sl(row, races), axis = 1)

split_precincts_list

{'Bartow-:-Cartersville West': {'SU': ['37', '52']},
 'Bibb-:-Howard 1': {'SU': ['18', '25']},
 'Bibb-:-Howard 5': {'SU': ['18', '25']},
 'Chatham-:-1-14C St. Luke United Methodist Church': {'SU': ['01', '02']},
 'Chatham-:-3-15C Eli Whitney Complex': {'SU': ['01', '02']},
 'Chatham-:-7-06C 1st Baptist Church of Pooler': {'SU': ['01', '04']},
 'Chatham-:-7-08C Bloomingdale Comm Ctr': {'SU': ['01', '04']},
 'Chatham-:-7-09C Compassion Christian, Henderson': {'SU': ['01', '02']},
 'Chatham-:-7-10C Progressive Rec Center': {'SU': ['01', '02']},
 'Chatham-:-7-12C West Chatham Baptist Church': {'SU': ['01', '04']},
 'Chatham-:-7-13C Southside Fire Trng Ctr': {'SU': ['02', '04']},
 'Chatham-:-7-14C Coastal Cathedral': {'SU': ['01', '02', '04']},
 'Chatham-:-7-16C Pooler Recreation Center Gymnasium': {'SU': ['01', '04']},
 'Clarke-:-3B': {'SU': ['46', '47']},
 'Clarke-:-6A': {'SU': ['46', '47']},
 'Clarke-:-6C': {'SU': ['46', '47']},
 'Cobb-:-Chestnut Ridge 01': {'SU': ['32', '56']},
 'Cobb-:

In [81]:
#Create Split GDF
sldu_w_splits = sldu.copy()
for val in sldu["UNIQUE_ID"]:
    sl_list = []
    if val in split_precincts_list.keys():
        sldu_w_splits = district_splits_mod(split_precincts_list[val],"SLDU",val, sldu_w_splits, ga_sldu, "UNIQUE_ID", "SS_DIST", races, "SLDU_DIST")

In [82]:
#check
set(sldu_w_splits['UNIQUE_ID']) - set(sldu['UNIQUE_ID'])

{'Bartow-:-Cartersville West-(SLDU-052)',
 'Bibb-:-Howard 1-(SLDU-018)',
 'Bibb-:-Howard 5-(SLDU-018)',
 'Bibb-:-Howard 5-(SLDU-025)',
 'Chatham-:-1-14C St. Luke United Methodist Church-(SLDU-001)',
 'Chatham-:-3-15C Eli Whitney Complex-(SLDU-001)',
 'Chatham-:-3-15C Eli Whitney Complex-(SLDU-002)',
 'Chatham-:-7-06C 1st Baptist Church of Pooler-(SLDU-004)',
 'Chatham-:-7-08C Bloomingdale Comm Ctr-(SLDU-001)',
 'Chatham-:-7-08C Bloomingdale Comm Ctr-(SLDU-002)',
 'Chatham-:-7-08C Bloomingdale Comm Ctr-(SLDU-004)',
 'Chatham-:-7-09C Compassion Christian, Henderson-(SLDU-001)',
 'Chatham-:-7-10C Progressive Rec Center-(SLDU-001)',
 'Chatham-:-7-10C Progressive Rec Center-(SLDU-002)',
 'Chatham-:-7-12C West Chatham Baptist Church-(SLDU-001)',
 'Chatham-:-7-12C West Chatham Baptist Church-(SLDU-004)',
 'Chatham-:-7-13C Southside Fire Trng Ctr-(SLDU-002)',
 'Chatham-:-7-13C Southside Fire Trng Ctr-(SLDU-004)',
 'Chatham-:-7-14C Coastal Cathedral-(SLDU-002)',
 'Chatham-:-7-16C Pooler Recreat

In [83]:
#fillna
sldu_w_splits = sldu_w_splits.fillna(0)
#Adjust precinct name field to account for name changes
sldu_w_splits['Precinct'] = sldu_w_splits['UNIQUE_ID'].apply(lambda x: x.split('-:-')[1])
#reorder df
sldu_w_splits = sldu_w_splits[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct', 'SLDU_DIST'] + sldu_cols + ['geometry']]
#check
sldu_w_splits[sldu_w_splits["SLDU_DIST"].isna()]

,UNIQUE_ID,COUNTYFP,county,precinct,SLDU_DIST,PSU01DJON,PSU01DNIQ,PSU01RWAT,PSU02RYOU,PSU02RYAS,PSU02DMAL,PSU02DSCO,PSU03RHOD,PSU03RJON,PSU03RHAY,PSU04RHIC,PSU05DRAH,PSU05DPOW,PSU06DEST,PSU06DWAK,PSU06RGLA,PSU06RMOO,PSU07RMCK,PSU07RSAN,PSU07DISL,PSU07DMOO,PSU08RGOO,PSU09DMER,PSU10DJON,PSU10DHOL,PSU11RBUR,PSU12DSIM,PSU13RSUM,PSU13RPER,PSU13RWOO,PSU14DMCL,PSU14RHAU,PSU15DHAR,PSU16RHAR,PSU16DDUB,PSU17RSTR,PSU17RMAU,PSU17DMOR,PSU18RKEN,PSU18RVAN,PSU18DBEN,PSU19RTIL,PSU19DMOO,PSU20RWAL,PSU21RBEA,PSU22RDAN,PSU22DJON,PSU23RBUR,PSU24RAND,PSU25DROD,PSU25RWIL,PSU25RJEF,PSU25ROLI,PSU25RSUL,PSU26DLUC,PSU27DBIN,PSU27RDOL,PSU28RBRA,PSU29DWRI,PSU29DMAN,PSU29RROB,PSU30RDUG,PSU31RANA,PSU32DBEN,PSU32RKIR,PSU32RSOH,PSU33DRHE,PSU33DHEM,PSU34RSMI,PSU34DSEA,PSU35DJAM,PSU35DJON,PSU35DGIL,PSU35DGLA,PSU36DORR,PSU36DSMI,PSU36DSTE,PSU36RHUE,PSU37RSET,PSU37RDEL,PSU37RJOH,PSU37DPAR,PSU37DNIC,PSU38DTAT,PSU38DBRA,PSU38DCAR,PSU38DPET,PSU39DHAL,PSU40RMCD,PSU40DHAR,PSU41DJAC,PSU41RJON,PSU42DPAR,PSU43RWIL,PSU43DAND,PSU43DLES,PSU44DDAV,PSU44DFLO,PSU45DJON,PSU45RDIX,PSU46DFER,PSU46RCOW,PSU47RGIN,PSU47RHAR,PSU47RCHA,PSU47DSCO,PSU48RSTI,PSU48RGRI,PSU48DUDD,PSU49DCOO,PSU49RECH,PSU49RGIB,PSU49RSTR,PSU50DWIL,PSU50RHAT,PSU51RGOO,PSU52RHUF,PSU52RLEW,PSU52RKEE,PSU53RMOO,PSU53RHEN,PSU54RPAY,PSU54RKEE,PSU55DBUT,PSU55DODI,PSU56DTHO,PSU56RALB,RSU03RHOD,RSU03RJON,geometry


#### State House Districts

In [84]:
races = sldl_cols
precinct_mapping_dict = {}
split_precincts_list = {}
for index,row in sldl.iterrows():
    precinct_list = []
    for contest in races:
        if(row[contest]!=0):
            precinct_info = get_level_dist(contest)
            if precinct_info not in precinct_list:
                precinct_list.append(get_level_dist(contest))
    is_split = is_split_precinct(precinct_list)
    if (is_split):
        split_precincts_list[row["UNIQUE_ID"]]=is_split
    precinct_mapping_dict[row["UNIQUE_ID"]]=precinct_list
    
#Standardize district format with name different from PBER to use later and distinguish when splits occur
ga_sldl['SH_DIST']=ga_sldl["DISTRICT"].astype(str).str.zfill(2)
sldl["SLDL_DIST"] = sldl.apply(lambda row: return_sl(row, races), axis = 1)

split_precincts_list

{'Appling-:-1C': {'SL': ['15', '17']},
 'Appling-:-3C': {'SL': ['15', '17']},
 'Appling-:-5B': {'SL': ['15', '17']},
 'Barrow-:-05': {'SL': ['11', '12']},
 'Barrow-:-16': {'SL': ['10', '11']},
 'Ben Hill-:-East': {'SL': ['14', '15']},
 'Bulloch-:-Hagin': {'SL': ['15', '16']},
 'Cherokee-:-Freehome': {'SL': ['02', '04']},
 'Clayton-:-Lovejoy 7': {'SL': ['07', '11']},
 'Cobb-:-Baker 01': {'SL': ['02', '03']},
 'Cobb-:-Birney 01': {'SL': ['03', '04']},
 'Cobb-:-Cheatham Hill 03': {'SL': ['03', '04']},
 'Cobb-:-Elizabeth 03': {'SL': ['03', '04']},
 'Cobb-:-Elizabeth 04': {'SL': ['03', '04']},
 'Cobb-:-Kennesaw 1A': {'SL': ['02', '03']},
 'Cobb-:-Lindley 01': {'SL': ['03', '04']},
 'Cobb-:-Marietta 1A': {'SL': ['03', '04']},
 'Cobb-:-Marietta 5A': {'SL': ['03', '04']},
 'Cobb-:-Marietta 6A': {'SL': ['03', '04']},
 'Cobb-:-Oregon 02': {'SL': ['03', '04']},
 'Cobb-:-Oregon 03': {'SL': ['03', '04']},
 'Cobb-:-Oregon 05': {'SL': ['03', '04']},
 'Cobb-:-Palmer 01': {'SL': ['02', '04']},
 'Cobb-:

In [85]:
sldl_w_splits = sldl.copy()
for val in sldl["UNIQUE_ID"]:
    sl_list = []
    if val in split_precincts_list.keys():
        sldl_w_splits = district_splits_mod(split_precincts_list[val],"SLDL",val, sldl_w_splits, ga_sldl, "UNIQUE_ID", "SH_DIST", races, "SLDL_DIST")

In [86]:
#check
set(sldl_w_splits['UNIQUE_ID']) - set(sldl['UNIQUE_ID'])

{'Appling-:-1C-(SLDL-157)',
 'Appling-:-1C-(SLDL-178)',
 'Appling-:-3C-(SLDL-157)',
 'Appling-:-3C-(SLDL-178)',
 'Appling-:-5B-(SLDL-157)',
 'Appling-:-5B-(SLDL-178)',
 'Barrow-:-05-(SLDL-119)',
 'Barrow-:-05-(SLDL-120)',
 'Barrow-:-16-(SLDL-104)',
 'Barrow-:-16-(SLDL-119)',
 'Ben Hill-:-East-(SLDL-148)',
 'Ben Hill-:-East-(SLDL-156)',
 'Bulloch-:-Hagin-(SLDL-159)',
 'Cherokee-:-Freehome-(SLDL-011)',
 'Cherokee-:-Freehome-(SLDL-021)',
 'Cherokee-:-Freehome-(SLDL-023)',
 'Cherokee-:-Freehome-(SLDL-024)',
 'Cherokee-:-Freehome-(SLDL-047)',
 'Clayton-:-Lovejoy 7-(SLDL-078)',
 'Clayton-:-Lovejoy 7-(SLDL-116)',
 'Cobb-:-Baker 01-(SLDL-022)',
 'Cobb-:-Baker 01-(SLDL-035)',
 'Cobb-:-Birney 01-(SLDL-041)',
 'Cobb-:-Cheatham Hill 03-(SLDL-034)',
 'Cobb-:-Cheatham Hill 03-(SLDL-037)',
 'Cobb-:-Cheatham Hill 03-(SLDL-041)',
 'Cobb-:-Elizabeth 03-(SLDL-044)',
 'Cobb-:-Elizabeth 04-(SLDL-037)',
 'Cobb-:-Elizabeth 04-(SLDL-043)',
 'Cobb-:-Kennesaw 1A-(SLDL-022)',
 'Cobb-:-Kennesaw 1A-(SLDL-035)',
 '

In [87]:
#fillna
sldl_w_splits = sldl_w_splits.fillna(0)
#Adjust precinct name field to account for name changes
sldl_w_splits['Precinct'] = sldl_w_splits['UNIQUE_ID'].apply(lambda x: x.split('-:-')[1])
#reorder df
sldl_w_splits = sldl_w_splits[['UNIQUE_ID', 'COUNTYFP', 'county', 'precinct', 'SLDL_DIST'] + sldl_cols + ['geometry']]
#check
sldl_w_splits[sldl_w_splits["SLDL_DIST"].isna()]

,UNIQUE_ID,COUNTYFP,county,precinct,SLDL_DIST,PSL001RCAM,PSL001RHAR,PSL002RTAR,PSL002RCOL,PSL002RNOB,PSL003RHOR,PSL003RWEL,PSL004RCAR,PSL004RVOY,PSL005RBAR,PSL006RRID,PSL006RCOK,PSL007RRAL,PSL008DKRI,PSL008RGUN,PSL009RWAD,PSL009RTOL,PSL010RAND,PSL011DHOL,PSL011RJAS,PSL012RLUM,PSL012RWAT,PSL013DDEM,PSL013RMAR,PSL013RBAR,PSL014RSCO,PSL015RGAM,PSL016RKEL,PSL016RRIC,PSL017RMOM,PSL017RWOL,PSL017DMAR,PSL018RSMI,PSL018DRHU,PSL019DCOY,PSL019RGUL,PSL020RBYR,PSL020RSAU,PSL020RHIX,PSL021RTHO,PSL022RRID,PSL022RKOS,PSL022DHIL,PSL023RBAL,PSL023RHUT,PSL024DWAL,PSL024RGIL,PSL024RBAR,PSL024RSOL,PSL025DMEY,PSL025RJON,PSL026RMCD,PSL026DHEL,PSL027RHAW,PSL028RCOX,PSL028RTRE,PSL028RSHO,PSL028RMCC,PSL028RLAN,PSL028RLUC,PSL028DWOO,PSL029RDUB,PSL029DPAN,PSL030DFLO,PSL030RMCC,PSL030RPIM,PSL030RSAN,PSL031RDUN,PSL031RCLE,PSL032RERW,PSL033RPOW,PSL033RPUR,PSL034DCOK,PSL034RSEA,PSL035DCAM,PSL035DRIN,PSL035DMIL,PSL035RTRI,PSL036DRYN,PSL036REHR,PSL037DWIL,PSL037RRED,PSL038DWIL,PSL039DDEL,PSL039DCUM,PSL039DPIE,PSL039DJOH,PSL039DLES,PSL039RANG,PSL040DSTO,PSL040DCAS,PSL040RFON,PSL041DSMI,PSL041RROD,PSL042DANU,PSL043RTIL,PSL043DADE,PSL043DSTA,PSL044RPAR,PSL044DOYO,PSL045DMCC,PSL045RCOO,PSL045RMOO,PSL046DGAR,PSL046RCAR,PSL047RJON,PSL048DROB,PSL048RHIL,PSL049DGIL,PSL049RMAR,PSL049RGOR,PSL050RKRA,PSL050RRED,PSL050RTRA,PSL050DAU,PSL051RKOR,PSL051DPAN,PSL051DBRU,PSL052DROB,PSL052RAHR,PSL053RSIL,PSL053DCOF,PSL054RBAI,PSL054DHOL,PSL055DWIL,PSL055DGRE,PSL055RLEN,PSL056DMAI,PSL056DJON,PSL056DCHA,PSL057DEVA,PSL058DCAN,PSL058DTON,PSL059DOLA,PSL059DCOL,PSL060DJON,PSL060DLEE,PSL061DBRU,PSL061DKEM,PSL061DDAW,PSL061DMCC,PSL062DMIL,PSL062DCAL,PSL062DNOB,PSL063DSCH,PSL064DEDW,PSL064DTHO,PSL064DWIL,PSL064RNEW,PSL064RMIL,PSL064RPAR,PSL065DTHO,PSL065RHOR,PSL066DALE,PSL067DGLA,PSL067RAMI,PSL068DNAG,PSL068DWIL,PSL068RMAT,PSL069DBAZ,PSL069DJAC,PSL069DBRO,PSL070DAND,PSL070RSMI,PSL070RNUN,PSL071DOKA,PSL071RCOL,PSL072RHUD,PSL073RBON,PSL074DHAR,PSL074DMIT,PSL074RMAT,PSL074RBAL,PSL075DGLA,PSL075DAND,PSL075RASH,PSL076DSCO,PSL077DBUR,PSL078DDOU,PSL078DJEA,PSL079DNEA,PSL080RAND,PSL080DTRA,PSL081RBEN,PSL081DHOL,PSL082DOLI,PSL082RMIL,PSL083DLUP,PSL083RBER,PSL084DCRA,PSL084DRAE,PSL085DDRE,PSL085DONE,PSL086DBAR,PSL086DADA,PSL086DIVE,PSL086RKIN,PSL087DDAV,PSL088RFRE,PSL088DMIT,PSL088DROG,PSL089DEVA,PSL089RSHE,PSL090RDIO,PSL090DDRA,PSL090DSCH,PSL090DHUD,PSL090DHUB,PSL090DPAR,PSL091DMOO,PSL091DSHE,PSL092DTAY,PSL092DKIN,PSL093DCAR,PSL093DIZZ,PSL094DBEN,PSL095RDAW,PSL095DKEN,PSL096DMAR,PSL096RLOW,PSL097DROM,PSL097DWU,PSL097RCHA,PSL098DLIM,PSL099DDUG,PSL099RREE,PSL100DJAC,PSL100RCLA,PSL100RRIC,PSL101RPRO,PSL101DKEN,PSL102DOKO,PSL102RHAR,PSL103RHON,PSL103DANA,PSL104REFS,PSL104DREI,PSL105RCHO,PSL105DMUG,PSL106RWRE,PSL106DHUT,PSL106DMIT,PSL107DPAR,PSL107RCAO,PSL108DCLA,PSL108RCRI,PSL109DMCC,PSL110DADE,PSL110RLOL,PSL111DCOX,PSL111RMAR,PSL112RWIL,PSL112DREE,PSL113DHEN,PSL113DBOY,PSL114DADA,PSL114RFLE,PSL114RMCN,PSL115DLEW,PSL116DHOL,PSL116RBEN,PSL117DRUC,PSL117DCUL,PSL117RDAN,PSL117RKAH,PSL117RCAR,PSL117RCLA,PSL118RCRO,PSL118DBEL,PSL119RRAM,PSL119RRAY,PSL120DJOH,PSL120RGAI,PSL121DAUE,PSL121RWIE,PSL122DFRY,PSL123RLEV,PSL124RRHO,PSL124DHOW,PSL125RFLE,PSL126DFRA,PSL126RHAR,PSL127RNEW,PSL128DJAC,PSL129DHOW,PSL130RSWE,PSL130DGLA,PSL131RLOT,PSL132DPRI,PSL132DGEO,PSL133DWAL,PSL133RVAN,PSL134RKNI,PSL134DDIC,PSL135RCAM,PSL135RBRU,PSL136RJEN,PSL136RMUR,PSL137RRIC,PSL137DBUC,PSL138RSMI,PSL139RSMI,PSL140DREE,PSL140DBAK,PSL141DHUG,PSL142DPAR,PSL143DBEV,PSL144RWAS,PSL144RSTO,PSL144DCON,PSL145RDIC,PSL146RBLA,PSL146DDRI,PSL147RBAL,PSL147DPHI,PSL148RWIL,PSL149RMAT,PSL149RPRU,PSL150DBEN,PSL151RCHE,PSL151DBAR,PSL152RYEA,PSL153DSAM,PSL153DWYN,PSL153RTAY,PSL154RGRE,PSL154DHAY,PSL155RHAT,PSL156RHAG,PSL156DKIT,PSL157RWER,PSL158DSMI,PSL158RPAR,PSL159RBUR,PSL160RFRA,PSL161RHIT,PSL161DBAR,PSL162DGIL,PSL162DPRA,PSL163DWES,PSL163DBOW,PSL164DTHO,PSL164RSTE,PSL164RPAW,PSL165DJAC,PSL166RPET,PSL167RDEL,PSL168DWIL,PSL168DSMI,PSL169RPIR,PSL169DBRO,PSL170RHOU,PSL171RCAM,PSL172RWAT,PSL173DJEN,PSL173RTAY,PSL174RCOR,PSL175RLAH,PSL175RBLA,PSL176R

<p><a name="check"></a></p>

### Vote Total Checks

In [88]:
#Statewide GDF
county_totals_check(ga_prim_final, "RDH raw", st, "split", st_cols, "county")

***Countywide Totals Check***

P22USSDWAR is equal across all counties
P22USSDJOH is equal across all counties
P22USSRWAL is equal across all counties
P22USSRBLA is equal across all counties
P22USSRSAD is equal across all counties
P22USSRCLA is equal across all counties
P22USSRKIN is equal across all counties
P22USSRMCC is equal across all counties
P22GOVDABR is equal across all counties
P22GOVRKEM is equal across all counties
P22GOVRPER is equal across all counties
P22GOVRTAY is equal across all counties
P22GOVRDAV is equal across all counties
P22GOVRWIL is equal across all counties
P22LTGDHAL is equal across all counties
P22LTGDBAI is equal across all counties
P22LTGDSHA is equal across all counties
P22LTGDBTY is equal across all counties
P22LTGDALL is equal across all counties
P22LTGDJAC is equal across all counties
P22LTGDBTO is equal across all counties
P22LTGDHAY is equal across all counties
P22LTGDMAL is equal across all counties
P22LTGRJON is equal across all counties
P22LTGRMI

In [89]:
#State house splits
county_totals_check(ga_prim_final, "RDH raw", sldl, "split", sldl_cols, "county")

***Countywide Totals Check***

PSL001RCAM is equal across all counties
PSL001RHAR is equal across all counties
PSL002RTAR is equal across all counties
PSL002RCOL is equal across all counties
PSL002RNOB is equal across all counties
PSL003RHOR is equal across all counties
PSL003RWEL is equal across all counties
PSL004RCAR is equal across all counties
PSL004RVOY is equal across all counties
PSL005RBAR is equal across all counties
PSL006RRID is equal across all counties
PSL006RCOK is equal across all counties
PSL007RRAL is equal across all counties
PSL008DKRI is equal across all counties
PSL008RGUN is equal across all counties
PSL009RWAD is equal across all counties
PSL009RTOL is equal across all counties
PSL010RAND is equal across all counties
PSL011DHOL is equal across all counties
PSL011RJAS is equal across all counties
PSL012RLUM is equal across all counties
PSL012RWAT is equal across all counties
PSL013DDEM is equal across all counties
PSL013RMAR is equal across all counties
PSL013RBA

PSL090DHUD is equal across all counties
PSL090DHUB is equal across all counties
PSL090DPAR is equal across all counties
PSL091DMOO is equal across all counties
PSL091DSHE is equal across all counties
PSL092DTAY is equal across all counties
PSL092DKIN is equal across all counties
PSL093DCAR is equal across all counties
PSL093DIZZ is equal across all counties
PSL094DBEN is equal across all counties
PSL095RDAW is equal across all counties
PSL095DKEN is equal across all counties
PSL096DMAR is equal across all counties
PSL096RLOW is equal across all counties
PSL097DROM is equal across all counties
PSL097DWU is equal across all counties
PSL097RCHA is equal across all counties
PSL098DLIM is equal across all counties
PSL099DDUG is equal across all counties
PSL099RREE is equal across all counties
PSL100DJAC is equal across all counties
PSL100RCLA is equal across all counties
PSL100RRIC is equal across all counties
PSL101RPRO is equal across all counties
PSL101DKEN is equal across all counties
P

In [90]:
#State Senate Splits
county_totals_check(ga_prim_final, "RDH raw", sldu, "split", sldu_cols, "county")

***Countywide Totals Check***

PSU01DJON is equal across all counties
PSU01DNIQ is equal across all counties
PSU01RWAT is equal across all counties
PSU02RYOU is equal across all counties
PSU02RYAS is equal across all counties
PSU02DMAL is equal across all counties
PSU02DSCO is equal across all counties
PSU03RHOD is equal across all counties
PSU03RJON is equal across all counties
PSU03RHAY is equal across all counties
PSU04RHIC is equal across all counties
PSU05DRAH is equal across all counties
PSU05DPOW is equal across all counties
PSU06DEST is equal across all counties
PSU06DWAK is equal across all counties
PSU06RGLA is equal across all counties
PSU06RMOO is equal across all counties
PSU07RMCK is equal across all counties
PSU07RSAN is equal across all counties
PSU07DISL is equal across all counties
PSU07DMOO is equal across all counties
PSU08RGOO is equal across all counties
PSU09DMER is equal across all counties
PSU10DJON is equal across all counties
PSU10DHOL is equal across all cou

In [91]:
#congressional splits
county_totals_check(ga_prim_final, "RDH raw", cong_w_splits, "split", cong_cols, "county")

***Countywide Totals Check***

PCON01RCAR is equal across all counties
PCON01DGRI is equal across all counties
PCON01DHER is equal across all counties
PCON01DMUN is equal across all counties
PCON02DBIS is equal across all counties
PCON02DOHA is equal across all counties
PCON02RHUN is equal across all counties
PCON02RWES is equal across all counties
PCON02RJOH is equal across all counties
PCON02RCHI is equal across all counties
PCON02RROB is equal across all counties
PCON02RWHI is equal across all counties
PCON03RFER is equal across all counties
PCON03RCRA is equal across all counties
PCON03DALM is equal across all counties
PCON04DJOH is equal across all counties
PCON04RCHA is equal across all counties
PCON04RIVY is equal across all counties
PCON05RZIM is equal across all counties
PCON05DWIL is equal across all counties
PCON05DSTO is equal across all counties
PCON05DMAC is equal across all counties
PCON06RMCC is equal across all counties
PCON06REVA contains differences in these counties

In the file with congressional splits, Cobb County and Henry County had differences in vote totals for the following candidates. These votes were dropped.

Cobb County
PCON06REVA - 1 vote
PCON06RHAR - 1 vote
PCON11DDAZ - 19 votes
PCON11RLOU - 32 votes
PCON13RHAW - 1 vote
PCON13DSCO - 2 votes
PCON13DFOR - 1 vote
RCON06RMCC - 1 vote

Henry County
PCON13RGON - 3 votes
PCON13DSCO - 3 votes
PCON13DDRI - 1 vote
PCON13DFOR - 1 vote

### Check Geoms

In [92]:
ga_prim_final['geometry'] = ga_prim_final.geometry.buffer(0)
st['geometry'] = st.geometry.buffer(0)
cong_w_splits['geometry'] = cong_w_splits.geometry.buffer(0)
sldu_w_splits['geometry'] = sldu_w_splits.geometry.buffer(0)
sldl_w_splits['geometry'] = sldl_w_splits.geometry.buffer(0)

<p><a name="exp"></a></p>

### Export Cleaned Precinct-Level Datasets

Export joined datasets

In [93]:
if not os.path.exists("./ga_2022_prim_pber/"):
    os.mkdir("./ga_2022_prim_pber/")
    os.mkdir("./ga_2022_prim_pber/ga_prim_22_no_splits_prec/")
    os.mkdir("./ga_2022_prim_pber/ga_prim_22_st_prec/")
    os.mkdir("./ga_2022_prim_pber/ga_prim_22_cong_prec/")
    os.mkdir("./ga_2022_prim_pber/ga_prim_22_sldu_prec/")
    os.mkdir("./ga_2022_prim_pber/ga_prim_22_sldl_prec/")

ga_prim_final.to_file("./ga_2022_prim_pber/ga_prim_22_no_splits_prec/ga_prim_22_no_splits_prec.shp")
st.to_file("./ga_2022_prim_pber/ga_prim_22_st_prec/ga_prim_22_st_prec.shp")
cong_w_splits.to_file("./ga_2022_prim_pber/ga_prim_22_cong_prec/ga_prim_22_cong_prec.shp")
sldu_w_splits.to_file("./ga_2022_prim_pber/ga_prim_22_sldu_prec/ga_prim_22_sldu_prec.shp")
sldl_w_splits.to_file("./ga_2022_prim_pber/ga_prim_22_sldl_prec/ga_prim_22_sldl_prec.shp")